In [ ]:
###########
# IMPORTS #
###########

!pip install openai
!pip install transformers
!pip install datasets
!pip install torchinfo

from google.colab import drive
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import pipeline
from torchinfo import summary
from tqdm import tqdm
#from urllib.parse import urlparse

import json
import math
import matplotlib.pyplot as plt
import numpy as np
import openai
import os
import pandas as pd
import random
import re
import seaborn as sn
import time
import torch
import xml.etree.ElementTree as ET

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
 from google.colab import drive
 drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#######
# GPU #
#######

# ! pip install accelerate
# from accelerate import Accelerator

# # use_fp16 = True
# use_fp16 = False

# if torch.cuda.is_available(): 
#     dev = "cuda:0"
#     accelerator = Accelerator(mixed_precision='fp16') if use_fp16 else Accelerator(mixed_precision='no')
# else:
#     dev = "cpu"
#     accelerator = Accelerator(mixed_precision='no')

# device = accelerator.device
# print('Using {}'.format(device))

In [ ]:
########
# TIME #
########

# start = time.time()

# # CODE TO BE TIMED #

# elapsed_time = time.time() - start

# et = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
# print(f"Elapsed time: {et}")

In [ ]:
###################
# SAVE/LOAD MODEL #
###################

# drive.mount('/content/drive')

# def save_model(path, model, step):
#     if os.path.exists(path):
#         save_model_path = path
#         torch.save({'step': step,
#                     'model_state_dict': model.cpu().state_dict()
#         }, save_model_path)

# def load_model(path, model):
#     if os.path.exists(path):
#         checkpoint = torch.load(path)
#         model.load_state_dict(checkpoint['model_state_dict'])

#     return model.to(device)

In [ ]:
################
# DATA CLASSES #
################

class ENEM_CLASS():
    def __init__(self, link):

        file_name = !basename {link}

        if not os.path.exists("/content/" + file_name[0]):
            ! wget -q -O {file_name[0]} {link}
            ! unzip -qq -o {file_name[0]}

        self.enem_version = []
        self.questions = []
        self.headers = []
        self.statements = []
        self.answers = []

        for filename in sorted(os.listdir('/content')):
            if filename.endswith(".xml"):

                # print(filename)
                # self.tree = ET.parse('2009-1.xml')

                tree = ET.parse(filename)
                root = tree.getroot()

                for question, header, statement, answers in zip(root.iter('question'),
                                                                root.iter('header'),
                                                                root.iter('statement'),
                                                                root.iter('answers')):
                    
                    # Based on the question dict, choose which questions will go into the data :
                    # {'CE': 'No', 'DS': 'No', 'EK': 'Yes', 'IC': 'No', 'MR': 'No', 'TC': 'Yes', 'id': '90', 'image': 'No'}
                    #if question.attrib['image'] == 'No': # or (question[''] == 'YES') ???
                    if question.attrib['IC'] == 'No' and  question.attrib['CE'] == 'No' and question.attrib['MR'] == 'No': 
                    #if True:

                        self.enem_version.append(os.path.splitext(filename)[0])

                        # Pergunta de imagens tem headers e alternativas vazias? - if acima exclui essas questões!
                        # header_text = re.sub(r'\n', '', header.text) if header.text else ""
                        # statement_text = re.sub(r'\n', '', statement.text) if statement.text else ""
                        # option_text = re.sub(r'\n', '', option.text) if option.text else ""

                        # Tirar \n no meio do texto
                        header_text = re.sub(r'\n', '', header.text)
                        statement_text = re.sub(r'\n', '', statement.text)

                        # Tirar espaços extras do início, fim e do meio ♫
                        header_text = re.sub(r"^\s+|\s+$", '', header_text)
                        header_text = re.sub(r"[\ ]{2}", ' ', header_text)

                        statement_text = re.sub(r"^\s+|\s+$", '', statement_text)
                        statement_text = re.sub(r"[\ ]{2}", ' ', statement_text)

                        self.questions.append(question.attrib)
                        self.headers.append(header_text)
                        self.statements.append(statement_text)

                        options = []
                        options_text = []

                        for option in list(answers):#answers.getchildren():

                            options.append(option.attrib)

                            # Tirar \n no meio do texto
                            option_text = re.sub(r'\n', '', option.text)

                            # Tirar espaços extras do início, fim e do meio ♫
                            option_text = re.sub(r"^\s+|\s+$", '', option_text)
                            option_text = re.sub(r"[\ ]{2}", ' ', option_text)

                            options_text.append(option_text)

                        self.answers.append((options, options_text))

    def __getitem__(self, idx):
        return self.enem_version[idx], self.questions[idx], self.headers[idx], self.statements[idx], self.answers[idx]

    def __len__(self):
        return len(self.questions)

    def print_question(self, idx):
        
        header = "Questão " + str(idx)
        print()
        print("#" * (len(header)+4))
        print(f'# {header} #')
        print("#" * (len(header)+4))
        print()

        print(f'Ano/Versão: {self.enem_version[idx]}\n')

        # print(f'Cabeçalho:\n{self.headers[idx]}\n')
        print('Cabeçalho:\n')
        header = self.headers[idx].split(' ')
        for i, tok in enumerate(header):
            if ((i % 15 == 0) and (i != 0)) and (i != (len(header)-1)):
                print(tok, end='\n')
            else:
                print(tok, end=' ')
        print('\n')
    
        # print(f'Enunciado:\n{self.statements[idx]}\n')
        print('Enunciado:\n')
        statement = self.statements[idx].split(' ')
        for i, tok in enumerate(statement):
            if ((i % 15 == 0) and (i != 0)) and (i != (len(statement)-1)):
                print(tok, end='\n')
            else:
                print(tok, end=' ')
        print('\n')

        print(f'Alternativas:')

        for alternative, answer in zip(self.answers[idx][0], self.answers[idx][1]):
            if (alternative['correct'] == 'No'):
                print(f" {alternative['id']}  - {answer}")
            else:
                print(f"[{alternative['id']}] - {answer}")

    def print_all_questions(self):
        for idx in range(len(self)):
            self.print_question(idx)
            print()

    def get_right_alternative(self, idx):
        for alternative, answer in zip(self.answers[idx][0], self.answers[idx][1]):
            if (alternative['correct'] == 'Yes'):
                return alternative['id'], answer
    
    def get_answers(self, idx):
        return list(zip(*ENEM.answers))[1][idx]

    def get_question_year(self, idx):
        return self.enem_version[idx]

class ENEM_data_CLASS(torch.utils.data.Dataset):
    def __init__(self, ENEM, tokenizer, max_length):
        super(ENEM_data_CLASS, self).__init__()

        self.ENEM = ENEM
        self.tokenizer = tokenizer
        self.max_length = max_length

        self.headers_tokenized = tokenizer(self.ENEM.headers, max_length = self.max_length, padding = True, truncation = True)
        self.statements_tokenized = tokenizer(self.ENEM.statements, max_length = self.max_length, padding = True, truncation = True)
        
        self.headers_ids = self.headers_tokenized['input_ids']
        self.headers_masks = self.headers_tokenized['attention_mask']

        self.statements_ids = self.statements_tokenized['input_ids']
        self.statements_masks = self.statements_tokenized['attention_mask']

        self.answers = []
        for idx in range(len(self.ENEM.answers)):
            options_text = list(zip(*ENEM.answers))[1][idx]
            options_tokenized = tokenizer(options_text, max_length = max_length, padding = True, truncation = True)

            options_ids = options_tokenized['input_ids']
            options_masks = options_tokenized['attention_mask']

            self.answers.append((options_ids, options_masks))

    def __len__(self):
        return len(self.ENEM)

    def __getitem__(self, idx):
        return (self.headers_ids[idx],self.headers_masks[idx]), (self.statements_ids[idx], self.statements_masks[idx]), (self.answers[idx][0], self.answers[idx][1])

In [ ]:
##############
# ENEM_CLASS #
##############

link = "https://www.ime.usp.br/~ddm/project/enem/ENEMdataset.zip"

ENEM = ENEM_CLASS(link)

print(f'Amount of questions in model: {len(ENEM)}')
#ENEM.print_all_questions()

# QUESTION 89 #

ENEM.print_question(89)

print()
print(f'Get all items from 89:')
print(ENEM.__getitem__(89))
print()
print(f'Get item 4 from 89:')
print(ENEM.__getitem__(89)[4])
print()
print(f'Get right answer from 89')
print(ENEM.get_right_alternative(89))

TypeError: ignored

In [ ]:
#########
# MODEL #
#########

model = AutoModelForPreTraining.from_pretrained('neuralmind/bert-base-portuguese-cased')
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)

In [ ]:
###################
# ENEM_DATA_CLASS #
###################

max_length = 512
batch_size = 32

ENEM_data = ENEM_data_CLASS(ENEM, tokenizer, max_length)

ENEM_data.answers

ENEM_dataloader = torch.utils.data.DataLoader(ENEM_data, batch_size=batch_size, drop_last = True, shuffle = True)

################
# TEST DECODER #
################

# ENEM.print_question(89)

# print()
# print(ENEM_data.__getitem__(89))
# print()

# print(tokenizer.decode([101,   123,  1803,   125,  6688,  1495,   117,   221,   179,   146,
#             2270,   366,  5341,   298,  5620,  1547,  9573,   117,  5084,   229,
#             1241,   984,   117,   834, 10144,   159,   259, 14642, 11335,   119,
#              102]))

In [ ]:
#@title Hello World to OpenAI

###############
# HELLO WORLD #
###############

# # Considerando a questão complete a alternativa correta
# # question = "Responda a seguinte pergunta de multipla escolha:\n\nA atmosfera  terrestre  é  composta  pelos  gases nitrogênio (N2) e oxigênio (O2), que somam cerca de 99%, e por gases traços, entre eles o gás carbônico (CO2), vapor de água (H2O), metano (CH4), ozônio (O3) e o óxido nitroso (N2O), que compõem o restante 1% do ar que respiramos. Os  gases  traços,  por  serem  constituídos  por  pelo  menos três  átomos,  conseguem  absorver  o  calor  irradiado  pela Terra, aquecendo o planeta. Esse fenômeno, que acontece há bilhões de anos, é chamado de efeito estufa. A partir da Revolução  Industrial  (século  XIX),  a  concentração  de gases  traços  na  atmosfera,  em  particular  o CO2,  tem aumentado significativamente, o que resultou no aumento da  temperatura  em  escala  global.  Mais  recentemente, outro fator tornou-se diretamente envolvido no aumento da concentração de CO2 na atmosfera: o desmatamento.  BROWN, I. F.; ALECHANDRE, A. S. Conceitos básicos sobre clima,  carbono, florestas e comunidades. A.G. Moreira & S.  Schwartzman. As mudanças climáticas globais e os  ecossistemas brasileiros. Brasília: Instituto de Pesquisa Ambiental da Amazônia, 2000 (adaptado).\n\nConsiderando  o texto,  uma  alternativa  viável  para combater o efeito estufa é:\n\nA. reduzir  o  calor irradiado  pela  Terra  mediante  a substituição da produção primária pela industrialização refrigerada.\nB. promover a queima da biomassa vegetal, responsável pelo  aumento  do  efeito  estufa  devido  à  produção  de CH4.\nC. reduzir  o  desmatamento,  mantendo-se,  assim,  o potencial  da  vegetação  em  absorver  o  CO2  da atmosfera.\nD. aumentar  a  concentração  atmosférica  de  H2O, molécula  capaz  de  absorver  grande  quantidade  de calor.\nE. remover  moléculas  orgânicas  polares  da  atmosfera, diminuindo a capacidade delas de reter calor.\n\nAlternativa correta: C. reduzir  o  desmatamento,  mantendo-se,  assim,  o potencial  da  vegetação  em  absorver  o  CO2  da atmosfera.\n\nResponda a seguinte pergunta de multipla escolha:\n\nEstima-se  que  haja  atualmente  no  mundo  40 milhões de pessoas infectadas pelo HIV (o vírus que causa a  AIDS),  sendo  que  as taxas  de  novas infecções continuam  crescendo,  principalmente  na  África,  Ásia  e Rússia. Nesse cenário de pandemia, uma vacina contra o HIV  teria  imenso  impacto,  pois  salvaria  milhões  de  vidas. Certamente  seria  um  marco  na  história  planetária  e também  uma  esperança  para  as  populações  carentes  de tratamento antiviral e de acompanhamento médico. TANURI, A.; FERREIRA JUNIOR, O. C. Vacina contra Aids: desafios e esperanças. Ciência Hoje (44) 26, 2009 (adaptado).\n\nUma vacina eficiente contra o HIV deveria:\n\nA. induzir  a  imunidade,  para  proteger  o  organismo  da contaminação viral.\nB. ser capaz de alterar o genoma do organismo portador, induzindo a síntese de enzimas protetoras.\nC. produzir  antígenos  capazes  de  se  ligarem  ao  vírus, impedindo  que  este  entre  nas  células  do  organismo humano.\nD. ser amplamente aplicada em animais, visto que esses são os principais transmissores do vírus para os seres humanos.\nE. estimular a imunidade, minimizando a transmissão do vírus por gotículas de saliva.\n\nAlternativa correta:\n"
# # question = "Responda a seguinte pergunta de multipla escolha:\n\nQual a capital do Brasil?\n\nA. A capital é Brasília.\nB. A capital é Buenos Aires.\nC. A capital é os Estados Unidos.\nD. A capital é Lima.\nE. A capital é Colombia.\n\nAlternativa correta: A. A capital é Brasília.\n\nResponda a seguinte pergunta de multipla escolha:\n\nQual a capital do Estados Unidos?\n\nA. A capital é Toronto.\nB. A capital é Lima.\nC. A capital é os Estados Unidos.\nD. A capital é Washington-DC.\nE. A capital é Buenos Aires.\n\nAlternativa correta:\n"

# question = "Responda a seguinte pergunta de multipla escolha:\n\nQual a capital da Argentina?\n\nA. A capital é Brasília.\nB. A capital é Buenos Aires.\nC. A capital é os Estados Unidos.\nD. A capital é Lima.\nE. A capital é Colombia.\n\nAlternativa correta:\n"

# alternative, text, alternatives_evaluation = OPENAI.prompt_openai(question)

# print(question)
# print(f'Alternativa correta do modelo: {alternative}')
# print(f'Text da alternativa do modelo: {text}')
# print()
# print(alternatives_evaluation)

In [ ]:
#@title 0-shot - v0 - WORKS
# ######################
# # OPENAI MODEL SETUP #
# ######################

# # BACKUP - PLAIN 0-SHOT

# class OPENAI_CLASS():
#     def __init__(self, api_key, model, ENEM):

#         self.ENEM = ENEM

#         self.model = model

#         self.openai = openai
#         self.openai.api_key = api_key

#     def get_alternatives_positions(self, text):

#         alternatives = []

#         # print(text)

#         for i in range(len(text)-1):

#             if ( re.findall("\n{1}", text[i-1]) and
#                  re.findall("^[ABCDE]{1}", text[i]) and
#                  re.findall("\.{1}", text[i+1]) ):

#                 j = i

#                 # print(f'len(text) = {len(text)} | i = {i} | text[i-1]: {repr(text[i-1])} | text[i]: {text[i]} | text[i+1]: {text[i+1]}')

#                 while text[j] != "\n" and text[j] != '<|endoftext|>':
#                     j += 1

#                 alternatives.append( {'start': i, 'end': j} )

#         return alternatives

#     def prepare_prompts(self, question):

#         alternatives = self.get_alternatives_positions(question)

#         text_to_prompt = []

#         for alt in alternatives:
#             # davinci-003 não fecha string
#             text_to_prompt.append(question + question[alt['start']:alt['end']] + '\n') # + '\n\n')

#         return text_to_prompt

#     def evaluate_responses(self, responses):

#         alternatives_evaluation = []

#         for response in responses:

#             tokens = response["choices"][0]["logprobs"]['tokens']
#             logprobs = response["choices"][0]["logprobs"]['token_logprobs']

#             alternatives = self.get_alternatives_positions(tokens)

#             alternatives_logprobs = []

#             for alt in alternatives:

#                 start = alt['start']
#                 end = alt['end']

#                 alt_logprobs = list(map(float, logprobs[start:end]))
#                 alt_n_toks = end - start
#                 prob = math.exp( (math.fsum(alt_logprobs) / alt_n_toks) )

#                 alternatives_logprobs.append( {'alternative': tokens[alt['start']],
#                                               'text':''.join(tokens[alt['start']:alt['end']]),
#                                               'prob': prob,
#                                               'temp_sum': math.fsum(alt_logprobs),
#                                               'temp_alt_n_toks': alt_n_toks} )

#             alternatives_evaluation.append(alternatives_logprobs[-1])

#         model_right_answer = max(alternatives_evaluation, key=lambda x:x['prob'])

#         return model_right_answer, alternatives_evaluation

#     def parse_question(self, idx):

#         instruction = "Responda a seguinte pergunta de multipla escolha:"
        
#         base_string = self.ENEM.headers[idx] + '\n\n' + self.ENEM.statements[idx] + '\n\n'

#         for alternative, answer in zip(self.ENEM.answers[idx][0], self.ENEM.answers[idx][1]):
#             base_string += alternative['id'] + ". " + answer + "\n"

#         prompt_string = instruction + "\n\n" +  base_string + "\n" + "Alternativa Correta:\n"

#         return prompt_string

#     def prompt_openai(self, question):

#         prompts = self.prepare_prompts(question)

#         print(f'prompt_openai: {prompts}')

#         responses = []

#         for text in tqdm(prompts):
#             response = openai.Completion.create(
#                 model=model,
#                 prompt=text,
#                 temperature=0,
#                 max_tokens=1,# TODO: 512? 2048? context?
#                 top_p=1,
#                 frequency_penalty=0,
#                 presence_penalty=0,
#                 echo=True,
#                 logprobs=5
#             )

#             responses.append(response)

#         model_right_answer, alternatives_evaluation = self.evaluate_responses(responses)

#         return model_right_answer['alternative'], model_right_answer['text'], alternatives_evaluation

#     def ask_ENEM_question(self, idx):

#         parsed_question = self.parse_question(idx)

#         alternative, text, alternatives_evaluation = self.prompt_openai(parsed_question)

#         return alternative, text, alternatives_evaluation, parsed_question

In [ ]:
#@title 0-shot / cot-few-shot - v1 - ABANDONADO
# ######################
# # OPENAI MODEL SETUP #
# ######################

# # RAWHIDE - CoT and 0-shot

# class OPENAI_CLASS():
#     def __init__(self, api_key, model, ENEM):

#         self.ENEM = ENEM

#         self.model = model

#         self.openai = openai
#         self.openai.api_key = api_key

#     def get_alternatives_positions(self, text):

#         # last_question_post only used on evaluate_responses()

#         ######################
#         # FIND LAST QUESTION #
#         ######################

#         # print(f'get_alternative_positions: {type(text)}')

#         shots = []

#         for i in range(len(text)):

#             if (type(text) == list):
#                 question = ''.join(text[i:i+19])
#             elif (type(text) == str):
#                 question = ''.join(text[i:i+51])

#             res = re.search(r'(Responda a seguinte pergunta de multipla escolha:\n\n)', question)

#             if (res != None):
#                 shots.append(i)

#         # print(f'shots: {shots}')
        
#         # print("#############################################")
#         # print(text)
#         # print(shots)
#         # print("#############################################")

#         last_question = text[shots[-1]:]

#         #print()
#         #print(shots[-1])
#         #print(f'last_question: {last_question}')
#         #print(text)

#         # print(last_question)
#         # print()

#         ###########################################
#         # FIND TOKENS POSITIONS FROM ALTERNATIVES #
#         ###########################################

#         alternatives = []

#         for i in range(len(last_question)-1):

#             if ( re.findall("\n{1}", last_question[i-1]) and
#                  re.findall("^[ABCDE]{1}", last_question[i]) and
#                  re.findall("\.{1}", last_question[i+1]) ):

#                 j = i

#                 # print(f'len(text) = {len(text)} | i = {i} | text[i-1]: {repr(text[i-1])} | text[i]: {text[i]} | text[i+1]: {text[i+1]}')
#                 # print(f'len(text) = {len(last_question)} | i = {i} | text[i-1]: {repr(last_question[i-1])} | text[i]: {last_question[i]} | text[i+1]: {last_question[i+1]}')

#                 while last_question[j] != "\n" and last_question[j] != '<|endoftext|>':
#                     # print(f"j: {j} | last_question: {last_question[j]}")
#                     j += 1

#                 alternatives.append( { 'start': (i + shots[-1]), 'end': (j + shots[-1]) } )

#         # for alt in alternatives:

#         #     start = alt['start']
#         #     end = alt['end']
            
#         #     print(text[start:end])
            
#         # print(desnes2)

#         # Beginning of last question on text, alternative positions from reference of last question
#         return shots[-1], alternatives

#     def evaluate_responses(self, last_question_pos, responses, CoT):

#         # print("evaluate_responses")

#         alternatives_evaluation = []

#         for response in responses:

#             tokens = response["choices"][0]["logprobs"]['tokens']
#             logprobs = response["choices"][0]["logprobs"]['token_logprobs']

#             last_question_pos, alternatives = self.get_alternatives_positions(tokens)

#             # if (CoT):
#             #     # Cut model generated AI answer
#             #     # tokens = tokens[0:alternatives[-1]['start']]
#             #     # tokens = tokens[0:alternatives[5]['end']]
#             #     alternatives = alternatives[0:6]

#             # print()
#             # print("$$$$$$$$$$$$$$")
#             # print(tokens)
#             # print(last_question_pos)
#             # print(alternatives)
#             # print(logprobs)
#             # print(len(alternatives))
#             # print("$$$$$$$$$$$$$$")
#             # print()

#             alternatives_logprobs = []

#             for alt in alternatives:

#                 start = alt['start']
#                 end = alt['end']

#                 if (tokens[end+1] == '<|endoftext|>'):
#                     end += 1

#                 alt_logprobs = list(map(float, logprobs[start:end]))
#                 alt_n_toks = end - start
#                 prob = math.exp( (math.fsum(alt_logprobs) / alt_n_toks) )

#                 alternatives_logprobs.append( {'alternative': tokens[alt['start']],
#                                               'text':''.join(tokens[alt['start']:alt['end']]),
#                                               'prob': prob,
#                                               'temp_sum': math.fsum(alt_logprobs),
#                                               'temp_alt_n_toks': alt_n_toks} )

#             alternatives_evaluation.append(alternatives_logprobs[-1])

#         model_right_answer = max(alternatives_evaluation, key=lambda x:x['prob'])

#         return model_right_answer, alternatives_evaluation

#     def prepare_prompts(self, question, CoT):

#         # print(f'prepare_prompts')

#         last_question_pos, alternatives = self.get_alternatives_positions(question)

#         if (CoT):
#             # Cut model generated AI answer
#             question = question[0:alternatives[-1]['start']]
#             alternatives = alternatives[0:5]

#             # print(temp[0:2610])
#             # print(alternatives[-1]['start'])

#         text_to_prompt = []

#         for alt in alternatives:
#             # davinci-003 não fecha string
#             # text_to_prompt.append(question + question[alt['start']:alt['end']] + '\n')
#             text_to_prompt.append(question + question[alt['start']:alt['end']] + '\n' + '<|endoftext|>')

#         return last_question_pos, text_to_prompt

#     def prompt_openai(self, question, CoT):

#         last_question_pos, prompts = self.prepare_prompts(question, CoT)

#         print(f'prompt_openai: {prompts}')
#         print()

#         responses = []

#         # for text in prompts:
#         for text in tqdm(prompts):
#             response = self.openai.Completion.create(
#                 model=self.model,
#                 prompt=text,
#                 temperature=0,
#                 max_tokens=1, # DON'T GENERATE TEXT
#                 top_p=1,
#                 frequency_penalty=0,
#                 presence_penalty=0,
#                 echo=True,
#                 logprobs=5
#             )

#             # print(response)
#             # print("#=============================#")
#             # print(response["choices"][0]['text'])
#             # print("#=============================#")
#             # print()
#             # time.sleep(60)

#             responses.append(response)

#         model_right_answer, alternatives_evaluation = self.evaluate_responses(last_question_pos, responses, CoT)

#         return model_right_answer['alternative'], model_right_answer['text'], alternatives_evaluation

#     def get_CoT_from_openai(self, first_question):
    
#         response = self.openai.Completion.create(
#             model=self.model,
#             prompt=first_question,
#             temperature=0,
#             max_tokens=1024, # https://beta.openai.com/docs/guides/completion/inserting-text
#             top_p=1,
#             frequency_penalty=0,
#             presence_penalty=0,
#             echo=True,
#             logprobs=5
#         )

#         reply = response["choices"][0]['text']

#         print("REPLY: ")
#         print(reply)

#         # print(desnes)

#         # Delimit close string from OpenAI
#         if (not re.findall("\n$", reply)):
#           reply = reply + '\n'

#         return reply

#     def parse_question(self, idx, CoT = ""):

#         instruction = "Responda a seguinte pergunta de multipla escolha:"
        
#         # base_string = self.ENEM.headers[idx] + '\n\n' + self.ENEM.statements[idx] + '\n\n'
#         base_string =  'Cabeçalho:\n' + self.ENEM.headers[idx] + '\n\n' + 'Enunciado:\n' + self.ENEM.statements[idx] + '\n\n' + 'Alternativas:\n'

#         for alternative, answer in zip(self.ENEM.answers[idx][0], self.ENEM.answers[idx][1]):
#             base_string += alternative['id'] + ". " + answer + "\n"

#         if (CoT):
#             prompt_string = CoT + "\n\n" + instruction + "\n\n" +  base_string + "\n" + "Explicação:\n"
#         else:
#             prompt_string = instruction + "\n\n" +  base_string + "\n" + "Alternativa Correta:\n"

#         return prompt_string

#     def ask_ENEM_question(self, idx, CoT = ""):

#         parsed_question = self.parse_question(idx, CoT)

#         if (CoT):
#             parsed_question = self.get_CoT_from_openai(parsed_question)

#         alternative, text, alternatives_evaluation = self.prompt_openai(parsed_question, CoT)

#         return alternative, text, alternatives_evaluation, parsed_question

###########
## TESTS ##
###########

# #print(ENEM.print_question(33))

# # cmd1 = "Responda a seguinte pergunta de multipla escolha assinalando a única alternativa correta e justifique"
# # exp1 = "A primeira alternativa está correta porque precisamos induzir imunidade de um orgamismo para protegê-lo contra futura infecções. A segunda alternativa está errada pois não tem como fazer mudanças no genoma de um organismo. A terceira alternativa está errada pois antígenos não se ligam ao vírus. A quarta alternativa está errada pois HIV também é transmitido em humanos. A quinta alternativas está errada pois HIV não se transmite por gotículas de saliva. Portanto, a alternativa correta é:\nA. induzir a imunidade, para proteger o organismo da contaminação viral."
# # cot1 = cmd1 + "\n\nCabeçalho:\nEstima-se que haja atualmente no mundo 40 milhões de pessoas infectadas pelo HIV (o vírus que causa a AIDS), sendo que as taxas de novas infecções continuam crescendo, principalmente na África, Ásia e Rússia. Nesse cenário de pandemia, uma vacina contra o HIV teria imenso impacto, pois salvaria milhões de vidas. Certamente seria um marco na história planetária e também uma esperança para as populações carentes de tratamento antiviral e de acompanhamento médico. TANURI, A.; FERREIRA JUNIOR, O. C. Vacina contra Aids: desafios e esperanças. Ciência Hoje (44) 26, 2009 (adaptado).\n\nEnunciado:\nConsiderando o texto, Uma vacina eficiente contra o HIV deveria:\n\nAlternativas:\nA. induzir a imunidade, para proteger o organismo da contaminação viral.\nB. ser capaz de alterar o genoma do organismo portador, induzindo a síntese de enzimas protetoras.\nC. produzir antígenos capazes de se ligarem ao vírus, impedindo que este entre nas células do organismo humano.\nD. ser amplamente aplicada em animais, visto que esses são os principais transmissores do vírus para os seres humanos.\nE. estimular a imunidade, minimizando a transmissão do vírus por gotículas de saliva.\n\nExplicação:\n" + exp1

# # Solução que proponho (ja está com 2 exemplos explicaçào e pergunta no final)

# cot3="Exemplo 1\n\nCabeçalho:\nSegundo Aristóteles, “na cidade com o melhor conjunto de normas e naquela dotada de homens absolutamente justos, os cidadãos não devem viver uma vida de trabalho trivial ou de negócios — esses tipos de vida são desprezíveis e incompatíveis com as qualidades morais —, tampouco devem ser agricultores os aspirantes à cidadania, pois o lazer é indispensável ao desenvolvimento das qualidades morais e à prática das atividades políticas”. VAN ACKER, T. Grécia. A vida cotidiana na cidade-Estado. São Paulo: Atual, 1994.\n\nEnunciado:\nO trecho, retirado da obra Política, de Aristóteles, permite compreender que a cidadania\n\nAlternativas:\nA. possui uma dimensão histórica que deve ser criticada, pois é condenável que os políticos de qualquer época fiquem entregues à ociosidade, enquanto o resto dos cidadãos tem de trabalhar.\nB. era entendida como uma dignidade própria dos grupos sociais superiores, fruto de uma concepção política profundamente hierarquizada da sociedade.\nC. estava vinculada, na Grécia Antiga, a uma percepção política democrática, que levava todos os habitantes da pólis a participarem da vida cívica.\nD. tinha profundas conexões com a justiça, razão pela qual o tempo livre dos cidadãos deveria ser dedicado às atividades vinculadas aos tribunais.\nE. vivida pelos atenienses era, de fato, restrita àqueles que se dedicavam à política e que tinham tempo para resolver os problemas da cidade.\n\nEXPLICAÇÃO:\nA alternativa A. está ERRADA. Porque faz um julgamento de valor de uma passagem histórica. Não responde o que é cidadania conforme o texto.\nA alternativa B. está CORRETA porque interpreta acertadamente a sentença estabelecida por Aristóteles, que vincula o direito à cidadania a um grupo seleto de indivíduos limitando o pleno gozo da cidadania restringindo a um seleto grupo de pessoas superiores \nA alternativa C. está ERRADA porque interpreta equivocadamente que todos habitantes teriam o mesmo direito cívico. O cabeçalho descreve que agricultores ou negociantes não deveriam ter o mesmo direito ao lazer como os políticos superiores. \nA alternativa D. está ERRADA porque interpreta equivocadamente o texto já que o cabeçalho não diz nada sobre tempo livre dos cidadãos e de que deveriam se dedicar aos tribunais\nA alternativa E. está ERRADA porque interpreta equivocadamente o texto dizendo que os que tinham direito a cidadania eram aqueles que tinham tempo para se dedicar a política. Mas isso não é mencionado no cabeçalho. Ao contrário o texto restringe a cidadania a um seleto grupo de pessoas superiores \n\nExemplo 2. \n\nCabeçalho:\nEstima-se que haja atualmente no mundo 40 milhões de pessoas infectadas pelo HIV (o vírus que causa a AIDS), sendo que as taxas de novas infecções continuam crescendo, principalmente na África, Ásia e Rússia. Nesse cenário de pandemia, uma vacina contra o HIV teria imenso impacto, pois salvaria milhões de vidas. Certamente seria um marco na história planetária e também uma esperança para as populações carentes de tratamento antiviral e de acompanhamento médico. TANURI, A.; FERREIRA JUNIOR, O. C. Vacina contra Aids: desafios e esperanças. Ciência Hoje (44) 26, 2009 (adaptado).\n\nEnunciado:\nConsiderando o texto, Uma vacina eficiente contra o HIV deveria:\n\nAlternativas:\nA. induzir a imunidade, para proteger o organismo da contaminação viral.\nB. ser capaz de alterar o genoma do organismo portador, induzindo a síntese de enzimas protetoras.\nC. produzir antígenos capazes de se ligarem ao vírus, impedindo que este entre nas células do organismo humano.\nD. ser amplamente aplicada em animais, visto que esses são os principais transmissores do vírus para os seres humanos.\nE. estimular a imunidade, minimizando a transmissão do vírus por gotículas de saliva.\n\nExplicação:\nA alternativa A. está CORRETA porque precisamos induzir imunidade de um organismo para protegê-lo contra futura infecções. \nA alternativa B. está ERRADA pois não é possível fazer mudanças no genoma de um organismo. \nA alternativa C. está ERRADA pois antígenos não se ligam ao vírus. \nA alternativa D está ERRADA pois pois HIV também é transmitido em humanos. \nA alternativa E está ERRADA pois HIV não se transmite por gotículas de saliva. \n\nQual dentre as cinco alternativas do Exemplo 3 A. B. C. D. ou E. está CORRETA ou ERRADA? Só é possível uma alternativa CORRETA. Complete a EXPLICAÇÃO \n\nExemplo 3\n"

# # Aquestão 38 é uma questào difícil que o zero shot tinha errado. Mas agora acerta !! 
# alternative, text, alternatives_evaluation, parsed_question = OPENAI.ask_ENEM_question(38, cot3)

# print(alternative)
# print(alternatives_evaluation)
# print(parsed_question)

In [ ]:
#@title 0-shot / cot-few-shot - v2 - WORKS
# ######################
# # OPENAI MODEL SETUP #
# ######################

# # RAWHIDE - CoT v2 and 0-shot

# class OPENAI_CLASS():
#     def __init__(self, api_key, model, ENEM):

#         self.ENEM = ENEM

#         self.model = model

#         self.openai = openai
#         self.openai.api_key = api_key

#     def get_alternatives_positions(self, question, pergunta_pos, cot = []):

#         #########
#         # SETUP #
#         #########

#         # REGEX few-shot:
#         # A alternativa (A) está ERRADA
        
#         # REGEX 0shot:
#         # B) ERRADA.
#         # (C) ERRADA.
        
#         # PS: Escape ()s on regex
#         correta_0sh = ['^', '\({0,1}', '[ABCDE]{1}', '\)', ' COR', 'RE', 'TA', '.']
#         correta_cot = ['^', 'A', ' altern', 'at', 'iva', ' \(', '[ABCDE]{1}', '\)', ' est', 'á', ' COR', 'RE', 'TA']

#         errada_0sh =  ['^', '\({0,1}', '[ABCDE]{1}', '\)', ' ER', 'R', 'ADA', '.']
#         errada_cot = ['^', 'A', ' altern', 'at', 'iva', ' \(', '[ABCDE]{1}', '\)', ' est', 'á', ' ER', 'R', 'ADA']

#         # TODO: investigate finding directly
#         # found_right = re.search(''.join(correta_cot), repr(question))
#         # print(found_right)

#         ###########################################
#         # FIND TOKENS POSITIONS FROM ALTERNATIVES #
#         ###########################################

#         found_right = None
#         found_wrong = None
        
#         alternatives = []

#         for i in range(len(question)-1):

#             if (cot):
#                 right = ''.join(question[i:i+len(correta_cot)])
#                 found_right = re.search(''.join(correta_cot), right)

#                 wrong = ''.join(question[i:i+len(errada_cot)])
#                 found_wrong = re.search(''.join(errada_cot), wrong)
#             else:
#                 right = ''.join(question[i:i+len(correta_0sh)])
#                 found_right = re.search(''.join(correta_0sh), right)

#                 wrong = ''.join(question[i:i+len(errada_0sh)])
#                 found_wrong = re.search(''.join(errada_0sh), wrong)

#             if ( found_right or found_wrong ):

#                 j = i

#                 while question[j] != "\n" and question[j] != '<|endoftext|>':
#                     j += 1

#                 alternatives.append( { 'start': (i + pergunta_pos), 'end': (j + pergunta_pos) } )

#                 found_right = None
#                 found_wrong = None

#         # POG
#         # Sometimes '(A) ERRADA' is matched twice due to (A) and A) - sometimes it isn't? WHY?
#         if (len(alternatives) == 10):
#             alternatives = alternatives[0:10:2]

#         # Alternatives positions from reference of last question
#         return alternatives

#     def evaluate_response(self, response, cot = []):

#         #########
#         # SETUP #
#         #########

#         # text = response["choices"][0]['text']
#         tokens = response["choices"][0]["logprobs"]['tokens']
#         logprobs = response["choices"][0]["logprobs"]['token_logprobs']

#         #################
#         # FIND PERGUNTA #
#         #################

#         pergunta = ['########', '####', '\n', '#', ' PER', 'G', 'UN', 'TA', ' #', '\n', '########', '####']
#         pergunta_pos = 0

#         for i in range(len(tokens)):

#             question = ''.join(tokens[i:i+len(pergunta)])
#             res = re.search(''.join(pergunta), question)

#             if (res != None):
#                 pergunta_pos = i

#         question = tokens[pergunta_pos:]

#         alternatives = self.get_alternatives_positions(question, pergunta_pos, cot)

#         alternatives_logprobs = []
#         alternatives_evaluation = []

#         for alt in alternatives:

#             start = alt['start']
#             end = alt['end']

#             alt_logprobs = list(map(float, logprobs[start:end]))
#             alt_n_toks = end - start
#             prob = math.exp( (math.fsum(alt_logprobs) / alt_n_toks) )

#             if(cot):
#                 # ['A', ' altern', 'at', 'iva', ' \(', '[ABCDE]{1}', '\)', ' est', 'á', ' COR', 'RE', 'TA]
#                 #                                          ^5
#                 alternatives_logprobs.append( {'alternative': tokens[start + 5],
#                                               'text':''.join(tokens[start:end]),
#                                               'prob': prob,
#                                               'temp_sum': math.fsum(alt_logprobs),
#                                               'temp_alt_n_toks': alt_n_toks} )
#             else:

#                 if ( tokens[start] == '(' ):
#                     # (C) ERRADA.
#                     #  ^
#                     _0sh_start = start + 1
#                 else:
#                     # C) ERRADA.
#                     # ^
#                     _0sh_start = start

#                 alternatives_logprobs.append( {'alternative': tokens[_0sh_start],
#                                               'text':''.join(tokens[start:end]),
#                                               'prob': prob,
#                                               'temp_sum': math.fsum(alt_logprobs),
#                                               'temp_alt_n_toks': alt_n_toks} )

#         model_right_answer = max(alternatives_logprobs, key=lambda x:x['prob'])

#         return model_right_answer, alternatives_logprobs

#     def ask_openai(self, prompt_string):
    
#         # print(repr(prompt_string))
#         # print(des)

#         response = self.openai.Completion.create(
#             model=self.model,
#             prompt=prompt_string,
#             temperature=0,
#             max_tokens=1024, # https://beta.openai.com/docs/guides/completion/inserting-text
#             top_p=1,
#             frequency_penalty=0,
#             presence_penalty=0,
#             echo=True,
#             logprobs=5
#         )

#         # reply = response["choices"][0]['text']
#         # print("REPLY: ")
#         # print(reply)
#         # print(desnes)

#         return response

#     def parse_question(self, idx, cmd, cot = []):

#         prompt_string = cmd + "\n"

#         # few-shot
#         for id, exemplo in enumerate(cot):
#             header = "EXEMPLO " + str(id+1)

#             prompt_string += "\n"
#             prompt_string += ("#" * (len(header)+4)) + "\n# " + header + " #\n" + ("#" * (len(header)+4))
#             prompt_string += "\n\n" + exemplo + "\n"
            
#         # print(prompt_string)
#         # print(des)

#         # 0-shot
#         header = "PERGUNTA"
        
#         prompt_string += "\n"
#         prompt_string +=  ("#" * (len(header)+4)) + "\n# " + header + " #\n" + ("#" * (len(header)+4))
#         prompt_string += "\n\n" + 'Cabeçalho:\n' + self.ENEM.headers[idx] + '\n\n' + 'Enunciado:\n' + self.ENEM.statements[idx] + '\n\n' + 'Alternativas:\n'

#         for alternative, answer in zip(self.ENEM.answers[idx][0], self.ENEM.answers[idx][1]):
#             prompt_string += '(' + alternative['id'] + ")" + " " + answer + "\n"

#         prompt_string += "\nExplicação:\n"

#         # print(prompt_string)
#         # print(len(prompt_string))
#         # print(des)

#         return prompt_string

#     def ask_ENEM_question(self, idx, cmd, cot = []):

#         parsed_question = self.parse_question(idx, cmd, cot)

#         openai_response = self.ask_openai(parsed_question)

#         model_right_answer, alternatives_evaluation = self.evaluate_response(openai_response, cot)

#         return parsed_question, openai_response, model_right_answer, alternatives_evaluation 

In [ ]:
#@title 0-shot / cot-few-shot - v3 - WORKS

######################
# OPENAI MODEL SETUP #
######################

# RAWHIDE - CoT v3 and 0-shot

class OPENAI_CLASS():
    def __init__(self, api_key, model, ENEM):

        self.ENEM = ENEM

        self.model = model

        self.openai = openai
        self.openai.api_key = api_key

    def get_alternatives_positions(self, tokens):

        ####################
        # FIND LINE BREAKS #
        ####################

        line_breaks = []

        for i, tok in enumerate(tokens):
            res = re.search('\n|<|endoftext|>', tok)
            if (res != None):
                line_breaks.append(i)

        #####################
        # FIND ALTERNATIVES #
        #####################

        alternatives = []

        # In multiline mode, ^ matches the position immediately following a newline
        # and $ matches the position immediately preceding a newline.
        regex_alt = ".*(CORRETA|ERRADA).*"
        regex_cmd = "^(Com base no cabeçalho e enunciado).*"

        for i in range(len(line_breaks)-1):
            
            string = "".join(tokens[line_breaks[i]+1:line_breaks[i+1]])

            found_alt = re.search(regex_alt, string)#, re.MULTILINE)
            found_cmd = re.search(regex_cmd, string)

            if ( (found_alt != None) and (found_cmd == None) ):
                alternatives.append( { 'start': (line_breaks[i]+1), 'end': (line_breaks[i+1]) } )

        return alternatives

    def evaluate_response(self, response, cot = []):

        #########
        # SETUP #
        #########

        # text = response["choices"][0]['text']
        tokens = response["choices"][0]["logprobs"]['tokens']
        logprobs = response["choices"][0]["logprobs"]['token_logprobs']

        # davinci-002 sometimes does not close the string at the end
        last_token = tokens[(len(tokens)-1)]
        if( last_token != '\n' or last_token != '<|endoftext|>'):
            tokens.append('\n')
            logprobs.append(0)

        alternatives = self.get_alternatives_positions(tokens)

        ################
        # ALTERNATIVES #
        ################

        alternatives_logprobs = []

        for alt in alternatives[-5:]:

            start = alt['start']
            end = alt['end']

            alternative_text = ''.join(tokens[start:end])

            alt_logprobs = list(map(float, logprobs[start:end]))
            alt_n_toks = end - start
            prob_logprobs = math.exp( (math.fsum(alt_logprobs) / alt_n_toks) )

            if "CORRETA" in alternative_text:
                score = prob_logprobs
            elif "ERRADA" in alternative_text:
                score = 1 - prob_logprobs

            # REGEX few-shot:
            # 1 - A alternativa (A) está ERRADA
            # 2 - A alternativa (A) bla bla bla. Portanto, a alternativa (A) está ERRADA.
            
            # REGEX 0shot:
            # 1 - B) ERRADA.
            # 2 - (C) ERRADA.

            if(cot):
                # ['A', ' altern', 'at', 'iva', ' \(', '[ABCDE]{1}', '\)', ' est', 'á', ' COR', 'RE', 'TA]
                #                                          ^5
                alternatives_logprobs.append( {'alternative': tokens[start + 5],
                                              'text':alternative_text,
                                              'prob': score,
                                              'temp_fsum': math.fsum(alt_logprobs),
                                              'temp_alt_n_toks': alt_n_toks} )
            else:

                if ( tokens[start] == '(' ):
                    # (C) ERRADA.
                    _0sh_start = start + 1
                else:
                    # C) ERRADA.
                    _0sh_start = start

                alternatives_logprobs.append( {'alternative': tokens[_0sh_start],
                                              'text':alternative_text,
                                              'prob': score,
                                              'temp_sum': math.fsum(alt_logprobs),
                                              'temp_alt_n_toks': alt_n_toks} )

        model_right_answer = max(alternatives_logprobs, key=lambda x:x['prob'])

        return model_right_answer, alternatives_logprobs

    def ask_openai(self, prompt_string):
    
        # print(repr(prompt_string))
        # print(des)

        response = self.openai.Completion.create(
            model=self.model,
            prompt=prompt_string,
            temperature=0,
            max_tokens=1024, # https://beta.openai.com/docs/guides/completion/inserting-text
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            echo=True,
            logprobs=5
        )

        # reply = response["choices"][0]['text']
        # print("\n###### REPLY ######\n")
        # print(reply)
        # print("\n###### REPLY ######\n")
        # print(desnes)

        return response

    # def parse_question(self, idx, cmd, cot = []):

    #     prompt_string = cmd + "\n"

    #     # few-shot
    #     for id, exemplo in enumerate(cot):
    #         header = "EXEMPLO " + str(id+1)

    #         prompt_string += "\n"
    #         prompt_string += ("#" * (len(header)+4)) + "\n# " + header + " #\n" + ("#" * (len(header)+4))
    #         prompt_string += "\n\n" + exemplo + "\n"
            
    #     # print(prompt_string)
    #     # print(des)

    #     # 0-shot
    #     header = "PERGUNTA"
        
    #     prompt_string += "\n"
    #     prompt_string +=  ("#" * (len(header)+4)) + "\n# " + header + " #\n" + ("#" * (len(header)+4))
    #     prompt_string += "\n\n" + 'Cabeçalho:\n' + self.ENEM.headers[idx] + '\n\n' + 'Enunciado:\n' + self.ENEM.statements[idx] + '\n\n' + 'Alternativas:\n'

    #     for alternative, answer in zip(self.ENEM.answers[idx][0], self.ENEM.answers[idx][1]):
    #         prompt_string += '(' + alternative['id'] + ")" + " " + answer + "\n"

    #     prompt_string += "\nExplicação:\n"

    #     # print(prompt_string)
    #     # print(len(prompt_string))
    #     # print(des)

    #     return prompt_string

    def parse_question(self, idx, cmd, cot = []):

        prompt_string = ""
        exemplo_id = 1000

        # few-shot
        for id, exemplo in enumerate(cot):
            prompt_string += "QUESTÃO " + str(exemplo_id) + "\n\n" + cmd + "\n\n" + exemplo + "\n\n"
            exemplo_id +=1

        # print(prompt_string)
        # print(des)

        # 0-shot
        prompt_string += "QUESTÃO " + str(idx) + "\n\n" + cmd + "\n\n" + 'Cabeçalho:\n' + self.ENEM.headers[idx] + '\n\n' + 'Enunciado:\n' + self.ENEM.statements[idx] + '\n\n' + 'Alternativas:\n'

        for alternative, answer in zip(self.ENEM.answers[idx][0], self.ENEM.answers[idx][1]):
            prompt_string += '(' + alternative['id'] + ")" + " " + answer + "\n"

        prompt_string += "\nExplicação:\n"

        # print(prompt_string)
        # print(len(prompt_string))
        # print(des)

        return prompt_string

    def ask_ENEM_question(self, idx, cmd, cot = []):

        parsed_question = self.parse_question(idx, cmd, cot)

        openai_response = self.ask_openai(parsed_question)

        model_right_answer, alternatives_evaluation = self.evaluate_response(openai_response, cot)

        return parsed_question, openai_response, model_right_answer, alternatives_evaluation 

In [ ]:
key = ""

# model = "text-davinci-002"
model = "text-davinci-003"
# model = "gpt-3.5-turbo"

OPENAI = OPENAI_CLASS(key, model, ENEM)

In [ ]:
#@title GPT3 devel [broken]
        # if (self.model == "gpt-3.5-turbo"):

        #     openai.Completion.create(model="gpt-3.5-turbo", prompt="Gimme logprobs", max_tokens=5, logprobs=3)

        #     response = self.openai.ChatCompletion.create(
        #         model=self.model,
        #         messages=[{"role": "user", "content": prompt_string}],
        #         temperature=0,
        #         max_tokens=1024, # https://beta.openai.com/docs/guides/completion/inserting-text
        #         top_p=1,
        #         frequency_penalty=0,
        #         presence_penalty=0,
        #         logprobs=5
        #     )

        #     # print(response)
        #     # print(desnes)

In [ ]:
#@title # FIRST IMPORTANT DEBUG CELL #

# DEBUG #

# CELL TO SEND TO OPENAI A PROMPT

#############
# 0 shot 11 #
#############

# BUG FREE:
# prompt_string = 'Com base no cabeçalho e enunciado, para cada uma das alternativas (A, B, C, D, E) decida se a pergunta está ERRADA ou CORRETA e dê uma explicação para cada alternativa. Apenas uma alternativa é CORRETA.\n\n############\n# PERGUNTA #\n############\n\nCabeçalho:\nCerca de 1% do lixo urbano é constituído por resíduos sólidos contendo elementos tóxicos. Entre esses elementos estão metais pesados como o cádmio, o chumbo e o mercúrio, componentes de pilhas e baterias, que são perigosos à saúde humana e ao meio ambiente. Quando descartadas em lixos comuns, pilhas e baterias vão para aterros sanitários ou lixões a céu aberto, e o vazamento de seus componentes contamina o solo, os rios e o lençol freático, atingindo a flora e a fauna. Por serem bioacumulativos e não biodegradáveis, esses metais chegam de forma acumulada aos seres humanos, por meio da cadeia alimentar. A legislação vigente (Resolução CONAMA no 257/1999) regulamenta o destino de pilhas e baterias após seu esgotamento energético e determina aos fabricantes e/ou importadores a quantidade máxima permitida desses metais em cada tipo de pilha/bateria, porém o problema ainda persiste. Disponível em: http://www.mma.gov.br. Acesso em: 11 jul. 2009 (adaptado).\n\nEnunciado:\nUma medida que poderia contribuir para acabar definitivamente com o problema da poluição ambiental por metais pesados relatado no texto seria\n\nAlternativas:\n(A) deixar de consumir aparelhos elétricos que utilizem pilha ou bateria como fonte de energia.\n(B) usar apenas pilhas ou baterias recarregáveis e de vida útil longa e evitar ingerir alimentos contaminados, especialmente peixes.\n(C) devolver pilhas e baterias, após o esgotamento da energia armazenada, à rede de assistência técnica especializada para repasse a fabricantes e/ou importadores.\n(D) criar nas cidades, especialmente naquelas com mais de 100 mil habitantes, pontos estratégicos de coleta de baterias e pilhas, para posterior repasse a fabricantes e/ou importadores.\n(E) exigir que fabricantes invistam em pesquisa para a substituição desses metais tóxicos por substâncias menos nocivas ao homem e ao ambiente, e que não sejam bioacumulativas.\n\nExplicação:\n'

###############
# few shot 11 #
###############

# BUG FREE:

# cot = []
# cot1 = "Cabeçalho:\nSegundo Aristóteles, “na cidade com o melhor conjunto de normas e naquela dotada de homens absolutamente justos, os cidadãos não devem viver uma vida de trabalho trivial ou de negócios — esses tipos de vida são desprezíveis e incompatíveis com as qualidades morais —, tampouco devem ser agricultores os aspirantes à cidadania, pois o lazer é indispensável ao desenvolvimento das qualidades morais e à prática das atividades políticas”. VAN ACKER, T. Grécia. A vida cotidiana na cidade-Estado. São Paulo: Atual, 1994.\n\nEnunciado:\nO trecho, retirado da obra Política, de Aristóteles, permite compreender que a cidadania\n\nAlternativas:\n(A) possui uma dimensão histórica que deve ser criticada, pois é condenável que os políticos de qualquer época fiquem entregues à ociosidade, enquanto o resto dos cidadãos tem de trabalhar.\n(B) era entendida como uma dignidade própria dos grupos sociais superiores, fruto de uma concepção política profundamente hierarquizada da sociedade.\n(C) estava vinculada, na Grécia Antiga, a uma percepção política democrática, que levava todos os habitantes da pólis a participarem da vida cívica.\n(D) tinha profundas conexões com a justiça, razão pela qual o tempo livre dos cidadãos deveria ser dedicado às atividades vinculadas aos tribunais.\n(E) vivida pelos atenienses era, de fato, restrita àqueles que se dedicavam à política e que tinham tempo para resolver os problemas da cidade.\n\nExplicação:\nA alternativa (A) está ERRADA porque faz um julgamento de valor de uma passagem histórica. Não responde o que é cidadania conforme o texto.\nA alternativa (B) está CORRETA porque interpreta acertadamente a sentença estabelecida por Aristóteles, que vincula o direito à cidadania a um grupo seleto de indivíduos limitando o pleno gozo da cidadania restringindo a um seleto grupo de pessoas superiores.\nA alternativa (C) está ERRADA porque interpreta equivocadamente que todos habitantes teriam o mesmo direito cívico. O cabeçalho descreve que agricultores ou negociantes não deveriam ter o mesmo direito ao lazer como os políticos superiores.\nA alternativa (D) está ERRADA porque interpreta equivocadamente o texto já que o cabeçalho não diz nada sobre tempo livre dos cidadãos e de que deveriam se dedicar aos tribunais.\nA alternativa (E) está ERRADA porque interpreta equivocadamente o texto dizendo que os que tinham direito a cidadania eram aqueles que tinham tempo para se dedicar a política. Mas isso não é mencionado no cabeçalho. Ao contrário o texto restringe a cidadania a um seleto grupo de pessoas superiores."
# cot2 = "Cabeçalho:\nEstima-se que haja atualmente no mundo 40 milhões de pessoas infectadas pelo HIV (o vírus que causa a AIDS), sendo que as taxas de novas infecções continuam crescendo, principalmente na África, Ásia e Rússia. Nesse cenário de pandemia, uma vacina contra o HIV teria imenso impacto, pois salvaria milhões de vidas. Certamente seria um marco na história planetária e também uma esperança para as populações carentes de tratamento antiviral e de acompanhamento médico. TANURI, A.; FERREIRA JUNIOR, O. C. Vacina contra Aids: desafios e esperanças. Ciência Hoje (44) 26, 2009 (adaptado).\n\nEnunciado:\nConsiderando o texto, uma vacina eficiente contra o HIV deveria:\n\nAlternativas:\n(A) induzir a imunidade, para proteger o organismo da contaminação viral.\n(B) ser capaz de alterar o genoma do organismo portador, induzindo a síntese de enzimas protetoras.\n(C) produzir antígenos capazes de se ligarem ao vírus, impedindo que este entre nas células do organismo humano.\n(D) ser amplamente aplicada em animais, visto que esses são os principais transmissores do vírus para os seres humanos.\n(E) estimular a imunidade, minimizando a transmissão do vírus por gotículas de saliva.\n\nExplicação:\nA alternativa (A) está CORRETA porque precisamos induzir imunidade de um organismo para protegê-lo contra futura infecções.\nA alternativa (B) está ERRADA pois não é possível fazer mudanças no genoma de um organismo.\nA alternativa (C) está ERRADA pois antígenos não se ligam ao vírus.\nA alternativa (D) está ERRADA pois pois HIV também é transmitido em humanos.\nA alternativa (E) está ERRADA pois HIV não se transmite por gotículas de saliva."

# cot.append(cot1)
# cot.append(cot2)

# prompt_string = 'Abaixo está uma lista de exemplos extraídos da prova do ENEM. Com base no cabeçalho e enunciado, para cada uma das alternativas (A, B, C, D, E) decida se a pergunta está ERRADA ou CORRETA e dê uma explicação para cada alternativa. Apenas uma alternativa é CORRETA.\n\n#############\n# EXEMPLO 1 #\n#############\n\nCabeçalho:\nSegundo Aristóteles, “na cidade com o melhor conjunto de normas e naquela dotada de homens absolutamente justos, os cidadãos não devem viver uma vida de trabalho trivial ou de negócios — esses tipos de vida são desprezíveis e incompatíveis com as qualidades morais —, tampouco devem ser agricultores os aspirantes à cidadania, pois o lazer é indispensável ao desenvolvimento das qualidades morais e à prática das atividades políticas”. VAN ACKER, T. Grécia. A vida cotidiana na cidade-Estado. São Paulo: Atual, 1994.\n\nEnunciado:\nO trecho, retirado da obra Política, de Aristóteles, permite compreender que a cidadania\n\nAlternativas:\n(A) possui uma dimensão histórica que deve ser criticada, pois é condenável que os políticos de qualquer época fiquem entregues à ociosidade, enquanto o resto dos cidadãos tem de trabalhar.\n(B) era entendida como uma dignidade própria dos grupos sociais superiores, fruto de uma concepção política profundamente hierarquizada da sociedade.\n(C) estava vinculada, na Grécia Antiga, a uma percepção política democrática, que levava todos os habitantes da pólis a participarem da vida cívica.\n(D) tinha profundas conexões com a justiça, razão pela qual o tempo livre dos cidadãos deveria ser dedicado às atividades vinculadas aos tribunais.\n(E) vivida pelos atenienses era, de fato, restrita àqueles que se dedicavam à política e que tinham tempo para resolver os problemas da cidade.\n\nExplicação:\nA alternativa (A) está ERRADA porque faz um julgamento de valor de uma passagem histórica. Não responde o que é cidadania conforme o texto.\nA alternativa (B) está CORRETA porque interpreta acertadamente a sentença estabelecida por Aristóteles, que vincula o direito à cidadania a um grupo seleto de indivíduos limitando o pleno gozo da cidadania restringindo a um seleto grupo de pessoas superiores.\nA alternativa (C) está ERRADA porque interpreta equivocadamente que todos habitantes teriam o mesmo direito cívico. O cabeçalho descreve que agricultores ou negociantes não deveriam ter o mesmo direito ao lazer como os políticos superiores.\nA alternativa (D) está ERRADA porque interpreta equivocadamente o texto já que o cabeçalho não diz nada sobre tempo livre dos cidadãos e de que deveriam se dedicar aos tribunais.\nA alternativa (E) está ERRADA porque interpreta equivocadamente o texto dizendo que os que tinham direito a cidadania eram aqueles que tinham tempo para se dedicar a política. Mas isso não é mencionado no cabeçalho. Ao contrário o texto restringe a cidadania a um seleto grupo de pessoas superiores.\n\n#############\n# EXEMPLO 2 #\n#############\n\nCabeçalho:\nEstima-se que haja atualmente no mundo 40 milhões de pessoas infectadas pelo HIV (o vírus que causa a AIDS), sendo que as taxas de novas infecções continuam crescendo, principalmente na África, Ásia e Rússia. Nesse cenário de pandemia, uma vacina contra o HIV teria imenso impacto, pois salvaria milhões de vidas. Certamente seria um marco na história planetária e também uma esperança para as populações carentes de tratamento antiviral e de acompanhamento médico. TANURI, A.; FERREIRA JUNIOR, O. C. Vacina contra Aids: desafios e esperanças. Ciência Hoje (44) 26, 2009 (adaptado).\n\nEnunciado:\nConsiderando o texto, uma vacina eficiente contra o HIV deveria:\n\nAlternativas:\n(A) induzir a imunidade, para proteger o organismo da contaminação viral.\n(B) ser capaz de alterar o genoma do organismo portador, induzindo a síntese de enzimas protetoras.\n(C) produzir antígenos capazes de se ligarem ao vírus, impedindo que este entre nas células do organismo humano.\n(D) ser amplamente aplicada em animais, visto que esses são os principais transmissores do vírus para os seres humanos.\n(E) estimular a imunidade, minimizando a transmissão do vírus por gotículas de saliva.\n\nExplicação:\nA alternativa (A) está CORRETA porque precisamos induzir imunidade de um organismo para protegê-lo contra futura infecções.\nA alternativa (B) está ERRADA pois não é possível fazer mudanças no genoma de um organismo.\nA alternativa (C) está ERRADA pois antígenos não se ligam ao vírus.\nA alternativa (D) está ERRADA pois pois HIV também é transmitido em humanos.\nA alternativa (E) está ERRADA pois HIV não se transmite por gotículas de saliva.\n\n############\n# PERGUNTA #\n############\n\nCabeçalho:\nCerca de 1% do lixo urbano é constituído por resíduos sólidos contendo elementos tóxicos. Entre esses elementos estão metais pesados como o cádmio, o chumbo e o mercúrio, componentes de pilhas e baterias, que são perigosos à saúde humana e ao meio ambiente. Quando descartadas em lixos comuns, pilhas e baterias vão para aterros sanitários ou lixões a céu aberto, e o vazamento de seus componentes contamina o solo, os rios e o lençol freático, atingindo a flora e a fauna. Por serem bioacumulativos e não biodegradáveis, esses metais chegam de forma acumulada aos seres humanos, por meio da cadeia alimentar. A legislação vigente (Resolução CONAMA no 257/1999) regulamenta o destino de pilhas e baterias após seu esgotamento energético e determina aos fabricantes e/ou importadores a quantidade máxima permitida desses metais em cada tipo de pilha/bateria, porém o problema ainda persiste. Disponível em: http://www.mma.gov.br. Acesso em: 11 jul. 2009 (adaptado).\n\nEnunciado:\nUma medida que poderia contribuir para acabar definitivamente com o problema da poluição ambiental por metais pesados relatado no texto seria\n\nAlternativas:\n(A) deixar de consumir aparelhos elétricos que utilizem pilha ou bateria como fonte de energia.\n(B) usar apenas pilhas ou baterias recarregáveis e de vida útil longa e evitar ingerir alimentos contaminados, especialmente peixes.\n(C) devolver pilhas e baterias, após o esgotamento da energia armazenada, à rede de assistência técnica especializada para repasse a fabricantes e/ou importadores.\n(D) criar nas cidades, especialmente naquelas com mais de 100 mil habitantes, pontos estratégicos de coleta de baterias e pilhas, para posterior repasse a fabricantes e/ou importadores.\n(E) exigir que fabricantes invistam em pesquisa para a substituição desses metais tóxicos por substâncias menos nocivas ao homem e ao ambiente, e que não sejam bioacumulativas.\n\nExplicação:\n'

#############
# 0 shot 13 #
#############

# BUG FREE:
# prompt_string = 'Com base no cabeçalho e enunciado, para cada uma das alternativas (A, B, C, D, E) decida se a pergunta está ERRADA ou CORRETA e dê uma explicação para cada alternativa. Apenas uma alternativa é CORRETA.\n\n############\n# PERGUNTA #\n############\n\nCabeçalho:\nO progresso da tecnologia introduziu diversos artefatos geradores de campos eletromagnéticos. Uma das mais empregadas invenções nessa área são os telefones celulares e smartphones. As tecnologias de transmissão de celular atualmente em uso no Brasil contemplam dois sistemas. O primeiro deles é operado entre as frequências de 800 MHz e 900 MHz e constitui os chamados sistemas TDMA/CDMA. Já a tecnologia GSM, ocupa a frequência de 1.800 MHz.\n\nEnunciado:\nConsiderando que a intensidade de transmissão e o nível de recepção “celular” sejam os mesmos para as tecnologias de transmissão TDMA/CDMA ou GSM, se um engenheiro tiver de escolher entre as duas tecnologias para obter a mesma cobertura, levando em consideração apenas o número de antenas em uma região, ele deverá escolher:\n\nAlternativas:\n(A) a tecnologia GSM, pois é a que opera com ondas de maior comprimento de onda.\n(B) a tecnologia TDMA/CDMA, pois é a que apresenta Efeito Doppler mais pronunciado.\n(C) a tecnologia GSM, pois é a que utiliza ondas que se propagam com maior velocidade.\n(D) qualquer uma das duas, pois as diferenças nas frequências são compensadas pelas diferenças nos comprimentos de onda.\n(E) qualquer uma das duas, pois nesse caso as intensidades decaem igualmente da mesma forma, independentemente da frequência.\n\nExplicação:\n'

###############
# few shot 13 #
###############

# BUG FREE:

# cot = []
# cot1 = "Cabeçalho:\nSegundo Aristóteles, “na cidade com o melhor conjunto de normas e naquela dotada de homens absolutamente justos, os cidadãos não devem viver uma vida de trabalho trivial ou de negócios — esses tipos de vida são desprezíveis e incompatíveis com as qualidades morais —, tampouco devem ser agricultores os aspirantes à cidadania, pois o lazer é indispensável ao desenvolvimento das qualidades morais e à prática das atividades políticas”. VAN ACKER, T. Grécia. A vida cotidiana na cidade-Estado. São Paulo: Atual, 1994.\n\nEnunciado:\nO trecho, retirado da obra Política, de Aristóteles, permite compreender que a cidadania\n\nAlternativas:\n(A) possui uma dimensão histórica que deve ser criticada, pois é condenável que os políticos de qualquer época fiquem entregues à ociosidade, enquanto o resto dos cidadãos tem de trabalhar.\n(B) era entendida como uma dignidade própria dos grupos sociais superiores, fruto de uma concepção política profundamente hierarquizada da sociedade.\n(C) estava vinculada, na Grécia Antiga, a uma percepção política democrática, que levava todos os habitantes da pólis a participarem da vida cívica.\n(D) tinha profundas conexões com a justiça, razão pela qual o tempo livre dos cidadãos deveria ser dedicado às atividades vinculadas aos tribunais.\n(E) vivida pelos atenienses era, de fato, restrita àqueles que se dedicavam à política e que tinham tempo para resolver os problemas da cidade.\n\nExplicação:\nA alternativa (A) está ERRADA porque faz um julgamento de valor de uma passagem histórica. Não responde o que é cidadania conforme o texto.\nA alternativa (B) está CORRETA porque interpreta acertadamente a sentença estabelecida por Aristóteles, que vincula o direito à cidadania a um grupo seleto de indivíduos limitando o pleno gozo da cidadania restringindo a um seleto grupo de pessoas superiores.\nA alternativa (C) está ERRADA porque interpreta equivocadamente que todos habitantes teriam o mesmo direito cívico. O cabeçalho descreve que agricultores ou negociantes não deveriam ter o mesmo direito ao lazer como os políticos superiores.\nA alternativa (D) está ERRADA porque interpreta equivocadamente o texto já que o cabeçalho não diz nada sobre tempo livre dos cidadãos e de que deveriam se dedicar aos tribunais.\nA alternativa (E) está ERRADA porque interpreta equivocadamente o texto dizendo que os que tinham direito a cidadania eram aqueles que tinham tempo para se dedicar a política. Mas isso não é mencionado no cabeçalho. Ao contrário o texto restringe a cidadania a um seleto grupo de pessoas superiores."
# cot2 = "Cabeçalho:\nEstima-se que haja atualmente no mundo 40 milhões de pessoas infectadas pelo HIV (o vírus que causa a AIDS), sendo que as taxas de novas infecções continuam crescendo, principalmente na África, Ásia e Rússia. Nesse cenário de pandemia, uma vacina contra o HIV teria imenso impacto, pois salvaria milhões de vidas. Certamente seria um marco na história planetária e também uma esperança para as populações carentes de tratamento antiviral e de acompanhamento médico. TANURI, A.; FERREIRA JUNIOR, O. C. Vacina contra Aids: desafios e esperanças. Ciência Hoje (44) 26, 2009 (adaptado).\n\nEnunciado:\nConsiderando o texto, uma vacina eficiente contra o HIV deveria:\n\nAlternativas:\n(A) induzir a imunidade, para proteger o organismo da contaminação viral.\n(B) ser capaz de alterar o genoma do organismo portador, induzindo a síntese de enzimas protetoras.\n(C) produzir antígenos capazes de se ligarem ao vírus, impedindo que este entre nas células do organismo humano.\n(D) ser amplamente aplicada em animais, visto que esses são os principais transmissores do vírus para os seres humanos.\n(E) estimular a imunidade, minimizando a transmissão do vírus por gotículas de saliva.\n\nExplicação:\nA alternativa (A) está CORRETA porque precisamos induzir imunidade de um organismo para protegê-lo contra futura infecções.\nA alternativa (B) está ERRADA pois não é possível fazer mudanças no genoma de um organismo.\nA alternativa (C) está ERRADA pois antígenos não se ligam ao vírus.\nA alternativa (D) está ERRADA pois pois HIV também é transmitido em humanos.\nA alternativa (E) está ERRADA pois HIV não se transmite por gotículas de saliva."

# cot.append(cot1)
# cot.append(cot2)

# prompt_string = 'Abaixo está uma lista de exemplos extraídos da prova do ENEM. Com base no cabeçalho e enunciado, para cada uma das alternativas (A, B, C, D, E) decida se a pergunta está ERRADA ou CORRETA e dê uma explicação para cada alternativa. Apenas uma alternativa é CORRETA.\n\n#############\n# EXEMPLO 1 #\n#############\n\nCabeçalho:\nSegundo Aristóteles, “na cidade com o melhor conjunto de normas e naquela dotada de homens absolutamente justos, os cidadãos não devem viver uma vida de trabalho trivial ou de negócios — esses tipos de vida são desprezíveis e incompatíveis com as qualidades morais —, tampouco devem ser agricultores os aspirantes à cidadania, pois o lazer é indispensável ao desenvolvimento das qualidades morais e à prática das atividades políticas”. VAN ACKER, T. Grécia. A vida cotidiana na cidade-Estado. São Paulo: Atual, 1994.\n\nEnunciado:\nO trecho, retirado da obra Política, de Aristóteles, permite compreender que a cidadania\n\nAlternativas:\n(A) possui uma dimensão histórica que deve ser criticada, pois é condenável que os políticos de qualquer época fiquem entregues à ociosidade, enquanto o resto dos cidadãos tem de trabalhar.\n(B) era entendida como uma dignidade própria dos grupos sociais superiores, fruto de uma concepção política profundamente hierarquizada da sociedade.\n(C) estava vinculada, na Grécia Antiga, a uma percepção política democrática, que levava todos os habitantes da pólis a participarem da vida cívica.\n(D) tinha profundas conexões com a justiça, razão pela qual o tempo livre dos cidadãos deveria ser dedicado às atividades vinculadas aos tribunais.\n(E) vivida pelos atenienses era, de fato, restrita àqueles que se dedicavam à política e que tinham tempo para resolver os problemas da cidade.\n\nExplicação:\nA alternativa (A) está ERRADA porque faz um julgamento de valor de uma passagem histórica. Não responde o que é cidadania conforme o texto.\nA alternativa (B) está CORRETA porque interpreta acertadamente a sentença estabelecida por Aristóteles, que vincula o direito à cidadania a um grupo seleto de indivíduos limitando o pleno gozo da cidadania restringindo a um seleto grupo de pessoas superiores.\nA alternativa (C) está ERRADA porque interpreta equivocadamente que todos habitantes teriam o mesmo direito cívico. O cabeçalho descreve que agricultores ou negociantes não deveriam ter o mesmo direito ao lazer como os políticos superiores.\nA alternativa (D) está ERRADA porque interpreta equivocadamente o texto já que o cabeçalho não diz nada sobre tempo livre dos cidadãos e de que deveriam se dedicar aos tribunais.\nA alternativa (E) está ERRADA porque interpreta equivocadamente o texto dizendo que os que tinham direito a cidadania eram aqueles que tinham tempo para se dedicar a política. Mas isso não é mencionado no cabeçalho. Ao contrário o texto restringe a cidadania a um seleto grupo de pessoas superiores.\n\n#############\n# EXEMPLO 2 #\n#############\n\nCabeçalho:\nEstima-se que haja atualmente no mundo 40 milhões de pessoas infectadas pelo HIV (o vírus que causa a AIDS), sendo que as taxas de novas infecções continuam crescendo, principalmente na África, Ásia e Rússia. Nesse cenário de pandemia, uma vacina contra o HIV teria imenso impacto, pois salvaria milhões de vidas. Certamente seria um marco na história planetária e também uma esperança para as populações carentes de tratamento antiviral e de acompanhamento médico. TANURI, A.; FERREIRA JUNIOR, O. C. Vacina contra Aids: desafios e esperanças. Ciência Hoje (44) 26, 2009 (adaptado).\n\nEnunciado:\nConsiderando o texto, uma vacina eficiente contra o HIV deveria:\n\nAlternativas:\n(A) induzir a imunidade, para proteger o organismo da contaminação viral.\n(B) ser capaz de alterar o genoma do organismo portador, induzindo a síntese de enzimas protetoras.\n(C) produzir antígenos capazes de se ligarem ao vírus, impedindo que este entre nas células do organismo humano.\n(D) ser amplamente aplicada em animais, visto que esses são os principais transmissores do vírus para os seres humanos.\n(E) estimular a imunidade, minimizando a transmissão do vírus por gotículas de saliva.\n\nExplicação:\nA alternativa (A) está CORRETA porque precisamos induzir imunidade de um organismo para protegê-lo contra futura infecções.\nA alternativa (B) está ERRADA pois não é possível fazer mudanças no genoma de um organismo.\nA alternativa (C) está ERRADA pois antígenos não se ligam ao vírus.\nA alternativa (D) está ERRADA pois pois HIV também é transmitido em humanos.\nA alternativa (E) está ERRADA pois HIV não se transmite por gotículas de saliva.\n\n############\n# PERGUNTA #\n############\n\nCabeçalho:\nO progresso da tecnologia introduziu diversos artefatos geradores de campos eletromagnéticos. Uma das mais empregadas invenções nessa área são os telefones celulares e smartphones. As tecnologias de transmissão de celular atualmente em uso no Brasil contemplam dois sistemas. O primeiro deles é operado entre as frequências de 800 MHz e 900 MHz e constitui os chamados sistemas TDMA/CDMA. Já a tecnologia GSM, ocupa a frequência de 1.800 MHz.\n\nEnunciado:\nConsiderando que a intensidade de transmissão e o nível de recepção “celular” sejam os mesmos para as tecnologias de transmissão TDMA/CDMA ou GSM, se um engenheiro tiver de escolher entre as duas tecnologias para obter a mesma cobertura, levando em consideração apenas o número de antenas em uma região, ele deverá escolher:\n\nAlternativas:\n(A) a tecnologia GSM, pois é a que opera com ondas de maior comprimento de onda.\n(B) a tecnologia TDMA/CDMA, pois é a que apresenta Efeito Doppler mais pronunciado.\n(C) a tecnologia GSM, pois é a que utiliza ondas que se propagam com maior velocidade.\n(D) qualquer uma das duas, pois as diferenças nas frequências são compensadas pelas diferenças nos comprimentos de onda.\n(E) qualquer uma das duas, pois nesse caso as intensidades decaem igualmente da mesma forma, independentemente da frequência.\n\nExplicação:\n'

#############
# 0 shot 89 #
#############

# prompt_string = 'Com base no cabeçalho e enunciado, para cada uma das alternativas (A, B, C, D, E) decida se a pergunta está ERRADA ou CORRETA e dê uma explicação para cada alternativa. Apenas uma alternativa é CORRETA.\n\n############\n# PERGUNTA #\n############\n\nCabeçalho:\nAntes, eram apenas as grandes cidades que se apresentavam como o império da técnica, objeto de modificações, suspensões, acréscimos, cada vez mais sofisticadas e carregadas de artifício. Esse mundo artificial inclui, hoje, o mundo rural. SANTOS, M. A Natureza do Espaço. São Paulo: Hucitec, 1996.\n\nEnunciado:\nConsiderando a transformação mencionada no texto, uma consequência socioespacial que caracteriza o atual mundo rural brasileiro é\n\nAlternativas:\n(A) a redução do processo de concentração de terras.\n(B) o aumento do aproveitamento de solos menos férteis.\n(C) a ampliação do isolamento do espaço rural.\n(D) a estagnação da fronteira agrícola do país.\n(E) a diminuição do nível de emprego formal.\n\nExplicação:\n'

###############
# few shot 89 #
###############

# BUG FREE:

# cot = []
# cot1 = "Cabeçalho:\nSegundo Aristóteles, “na cidade com o melhor conjunto de normas e naquela dotada de homens absolutamente justos, os cidadãos não devem viver uma vida de trabalho trivial ou de negócios — esses tipos de vida são desprezíveis e incompatíveis com as qualidades morais —, tampouco devem ser agricultores os aspirantes à cidadania, pois o lazer é indispensável ao desenvolvimento das qualidades morais e à prática das atividades políticas”. VAN ACKER, T. Grécia. A vida cotidiana na cidade-Estado. São Paulo: Atual, 1994.\n\nEnunciado:\nO trecho, retirado da obra Política, de Aristóteles, permite compreender que a cidadania\n\nAlternativas:\n(A) possui uma dimensão histórica que deve ser criticada, pois é condenável que os políticos de qualquer época fiquem entregues à ociosidade, enquanto o resto dos cidadãos tem de trabalhar.\n(B) era entendida como uma dignidade própria dos grupos sociais superiores, fruto de uma concepção política profundamente hierarquizada da sociedade.\n(C) estava vinculada, na Grécia Antiga, a uma percepção política democrática, que levava todos os habitantes da pólis a participarem da vida cívica.\n(D) tinha profundas conexões com a justiça, razão pela qual o tempo livre dos cidadãos deveria ser dedicado às atividades vinculadas aos tribunais.\n(E) vivida pelos atenienses era, de fato, restrita àqueles que se dedicavam à política e que tinham tempo para resolver os problemas da cidade.\n\nExplicação:\nA alternativa (A) está ERRADA porque faz um julgamento de valor de uma passagem histórica. Não responde o que é cidadania conforme o texto.\nA alternativa (B) está CORRETA porque interpreta acertadamente a sentença estabelecida por Aristóteles, que vincula o direito à cidadania a um grupo seleto de indivíduos limitando o pleno gozo da cidadania restringindo a um seleto grupo de pessoas superiores.\nA alternativa (C) está ERRADA porque interpreta equivocadamente que todos habitantes teriam o mesmo direito cívico. O cabeçalho descreve que agricultores ou negociantes não deveriam ter o mesmo direito ao lazer como os políticos superiores.\nA alternativa (D) está ERRADA porque interpreta equivocadamente o texto já que o cabeçalho não diz nada sobre tempo livre dos cidadãos e de que deveriam se dedicar aos tribunais.\nA alternativa (E) está ERRADA porque interpreta equivocadamente o texto dizendo que os que tinham direito a cidadania eram aqueles que tinham tempo para se dedicar a política. Mas isso não é mencionado no cabeçalho. Ao contrário o texto restringe a cidadania a um seleto grupo de pessoas superiores."
# cot2 = "Cabeçalho:\nEstima-se que haja atualmente no mundo 40 milhões de pessoas infectadas pelo HIV (o vírus que causa a AIDS), sendo que as taxas de novas infecções continuam crescendo, principalmente na África, Ásia e Rússia. Nesse cenário de pandemia, uma vacina contra o HIV teria imenso impacto, pois salvaria milhões de vidas. Certamente seria um marco na história planetária e também uma esperança para as populações carentes de tratamento antiviral e de acompanhamento médico. TANURI, A.; FERREIRA JUNIOR, O. C. Vacina contra Aids: desafios e esperanças. Ciência Hoje (44) 26, 2009 (adaptado).\n\nEnunciado:\nConsiderando o texto, uma vacina eficiente contra o HIV deveria:\n\nAlternativas:\n(A) induzir a imunidade, para proteger o organismo da contaminação viral.\n(B) ser capaz de alterar o genoma do organismo portador, induzindo a síntese de enzimas protetoras.\n(C) produzir antígenos capazes de se ligarem ao vírus, impedindo que este entre nas células do organismo humano.\n(D) ser amplamente aplicada em animais, visto que esses são os principais transmissores do vírus para os seres humanos.\n(E) estimular a imunidade, minimizando a transmissão do vírus por gotículas de saliva.\n\nExplicação:\nA alternativa (A) está CORRETA porque precisamos induzir imunidade de um organismo para protegê-lo contra futura infecções.\nA alternativa (B) está ERRADA pois não é possível fazer mudanças no genoma de um organismo.\nA alternativa (C) está ERRADA pois antígenos não se ligam ao vírus.\nA alternativa (D) está ERRADA pois pois HIV também é transmitido em humanos.\nA alternativa (E) está ERRADA pois HIV não se transmite por gotículas de saliva."

# cot.append(cot1)
# cot.append(cot2)

# prompt_string = 'Abaixo está uma lista de exemplos extraídos da prova do ENEM. Com base no cabeçalho e enunciado, para cada uma das alternativas (A, B, C, D, E) decida se a pergunta está ERRADA ou CORRETA e dê uma explicação para cada alternativa. Apenas uma alternativa é CORRETA.\n\n#############\n# EXEMPLO 1 #\n#############\n\nCabeçalho:\nSegundo Aristóteles, “na cidade com o melhor conjunto de normas e naquela dotada de homens absolutamente justos, os cidadãos não devem viver uma vida de trabalho trivial ou de negócios — esses tipos de vida são desprezíveis e incompatíveis com as qualidades morais —, tampouco devem ser agricultores os aspirantes à cidadania, pois o lazer é indispensável ao desenvolvimento das qualidades morais e à prática das atividades políticas”. VAN ACKER, T. Grécia. A vida cotidiana na cidade-Estado. São Paulo: Atual, 1994.\n\nEnunciado:\nO trecho, retirado da obra Política, de Aristóteles, permite compreender que a cidadania\n\nAlternativas:\n(A) possui uma dimensão histórica que deve ser criticada, pois é condenável que os políticos de qualquer época fiquem entregues à ociosidade, enquanto o resto dos cidadãos tem de trabalhar.\n(B) era entendida como uma dignidade própria dos grupos sociais superiores, fruto de uma concepção política profundamente hierarquizada da sociedade.\n(C) estava vinculada, na Grécia Antiga, a uma percepção política democrática, que levava todos os habitantes da pólis a participarem da vida cívica.\n(D) tinha profundas conexões com a justiça, razão pela qual o tempo livre dos cidadãos deveria ser dedicado às atividades vinculadas aos tribunais.\n(E) vivida pelos atenienses era, de fato, restrita àqueles que se dedicavam à política e que tinham tempo para resolver os problemas da cidade.\n\nExplicação:\nA alternativa (A) está ERRADA porque faz um julgamento de valor de uma passagem histórica. Não responde o que é cidadania conforme o texto.\nA alternativa (B) está CORRETA porque interpreta acertadamente a sentença estabelecida por Aristóteles, que vincula o direito à cidadania a um grupo seleto de indivíduos limitando o pleno gozo da cidadania restringindo a um seleto grupo de pessoas superiores.\nA alternativa (C) está ERRADA porque interpreta equivocadamente que todos habitantes teriam o mesmo direito cívico. O cabeçalho descreve que agricultores ou negociantes não deveriam ter o mesmo direito ao lazer como os políticos superiores.\nA alternativa (D) está ERRADA porque interpreta equivocadamente o texto já que o cabeçalho não diz nada sobre tempo livre dos cidadãos e de que deveriam se dedicar aos tribunais.\nA alternativa (E) está ERRADA porque interpreta equivocadamente o texto dizendo que os que tinham direito a cidadania eram aqueles que tinham tempo para se dedicar a política. Mas isso não é mencionado no cabeçalho. Ao contrário o texto restringe a cidadania a um seleto grupo de pessoas superiores.\n\n#############\n# EXEMPLO 2 #\n#############\n\nCabeçalho:\nEstima-se que haja atualmente no mundo 40 milhões de pessoas infectadas pelo HIV (o vírus que causa a AIDS), sendo que as taxas de novas infecções continuam crescendo, principalmente na África, Ásia e Rússia. Nesse cenário de pandemia, uma vacina contra o HIV teria imenso impacto, pois salvaria milhões de vidas. Certamente seria um marco na história planetária e também uma esperança para as populações carentes de tratamento antiviral e de acompanhamento médico. TANURI, A.; FERREIRA JUNIOR, O. C. Vacina contra Aids: desafios e esperanças. Ciência Hoje (44) 26, 2009 (adaptado).\n\nEnunciado:\nConsiderando o texto, uma vacina eficiente contra o HIV deveria:\n\nAlternativas:\n(A) induzir a imunidade, para proteger o organismo da contaminação viral.\n(B) ser capaz de alterar o genoma do organismo portador, induzindo a síntese de enzimas protetoras.\n(C) produzir antígenos capazes de se ligarem ao vírus, impedindo que este entre nas células do organismo humano.\n(D) ser amplamente aplicada em animais, visto que esses são os principais transmissores do vírus para os seres humanos.\n(E) estimular a imunidade, minimizando a transmissão do vírus por gotículas de saliva.\n\nExplicação:\nA alternativa (A) está CORRETA porque precisamos induzir imunidade de um organismo para protegê-lo contra futura infecções.\nA alternativa (B) está ERRADA pois não é possível fazer mudanças no genoma de um organismo.\nA alternativa (C) está ERRADA pois antígenos não se ligam ao vírus.\nA alternativa (D) está ERRADA pois pois HIV também é transmitido em humanos.\nA alternativa (E) está ERRADA pois HIV não se transmite por gotículas de saliva.\n\n############\n# PERGUNTA #\n############\n\nCabeçalho:\nAntes, eram apenas as grandes cidades que se apresentavam como o império da técnica, objeto de modificações, suspensões, acréscimos, cada vez mais sofisticadas e carregadas de artifício. Esse mundo artificial inclui, hoje, o mundo rural. SANTOS, M. A Natureza do Espaço. São Paulo: Hucitec, 1996.\n\nEnunciado:\nConsiderando a transformação mencionada no texto, uma consequência socioespacial que caracteriza o atual mundo rural brasileiro é\n\nAlternativas:\n(A) a redução do processo de concentração de terras.\n(B) o aumento do aproveitamento de solos menos férteis.\n(C) a ampliação do isolamento do espaço rural.\n(D) a estagnação da fronteira agrícola do país.\n(E) a diminuição do nível de emprego formal.\n\nExplicação:\n'

###########
# OPEN AI #
###########

# response = openai.Completion.create(
#     model=model,
#     prompt=prompt_string,
#     temperature=0,
#     max_tokens=1024, # https://beta.openai.com/docs/guides/completion/inserting-text
#     top_p=1,
#     frequency_penalty=0,
#     presence_penalty=0,
#     echo=True,
#     logprobs=5
# )

In [ ]:
#@title # IMPORTANT DEBUG CELL 2 #

# def get_alternatives_positions(question, pergunta_pos, cot = []):

#     #########
#     # SETUP #
#     #########

#     # few-shot:
#     # A alternativa (A) está ERRADA
    
#     # 0shot:
#     # B) ERRADA.
#     # (C) ERRADA.
    
#     # PS: Escape ()s on regex
#     # correta_0sh = ['\n', '\({0,1}', '[ABCDE]{1}', '\)', ' COR', 'RE', 'TA', '.']
#     # correta_cot = ['\n', 'A', ' altern', 'at', 'iva', ' \(', '[ABCDE]{1}', '\)', ' est', 'á', ' COR', 'RE', 'TA']

#     # errada_0sh =  ['\n', '\({0,1}', '[ABCDE]{1}', '\)', ' ER', 'R', 'ADA', '.']
#     # errada_cot = ['\n', 'A', ' altern', 'at', 'iva', ' \(', '[ABCDE]{1}', '\)', ' est', 'á', ' ER', 'R', 'ADA']

#     correta_0sh = ['^', '\({0,1}', '[ABCDE]{1}', '\)', ' COR', 'RE', 'TA', '.']
#     correta_cot = ['^', 'A', ' altern', 'at', 'iva', ' \(', '[ABCDE]{1}', '\)', ' est', 'á', ' COR', 'RE', 'TA']

#     errada_0sh =  ['^', '\({0,1}', '[ABCDE]{1}', '\)', ' ER', 'R', 'ADA', '.']
#     errada_cot = ['^', 'A', ' altern', 'at', 'iva', ' \(', '[ABCDE]{1}', '\)', ' est', 'á', ' ER', 'R', 'ADA']

#     # TODO: investigate finding directly
#     # found_right = re.search(''.join(correta_cot), repr(question))
#     # print(found_right)

#     ###########################################
#     # FIND TOKENS POSITIONS FROM ALTERNATIVES #
#     ###########################################

#     found_right = None
#     found_wrong = None
    
#     alternatives = []

#     for i in range(len(question)-1):

#         # print(len(alternatives))

#         # if (len(alternatives) == 1):
#         #     break

#         if (cot):
#             right = ''.join(question[i:i+len(correta_cot)])
#             found_right = re.search(''.join(correta_cot), right)

#             wrong = ''.join(question[i:i+len(errada_cot)])
#             found_wrong = re.search(''.join(errada_cot), wrong)
#         else:
#             right = ''.join(question[i:i+len(correta_0sh)])
#             found_right = re.search(''.join(correta_0sh), right)

#             wrong = ''.join(question[i:i+len(errada_0sh)])
#             found_wrong = re.search(''.join(errada_0sh), wrong)

# #        print("".join(correta_cot))
#         # print(len(correta_cot))
#         # print(len("".join(correta_cot)))
#         # print(repr(right))

#         if ( found_right or found_wrong ):

#             # print("FOUND SOMETHING")

#             # if(found_right != None):
#             #     print(found_right)

#             # if(found_wrong != None):
#             #     print(found_wrong)

#             j = i

#             # j+1 because of alternatives starting with \n
#             #while question[j+1] != "\n" and question[j+1] != '<|endoftext|>':
#             while question[j] != "\n" and question[j] != '<|endoftext|>':
#                 j += 1

#             alternatives.append( { 'start': (i + pergunta_pos), 'end': (j + pergunta_pos) } )
#             # alternatives.append( { 'start': (i), 'end': (j) } )

#             # print(repr("".join(question[i:i+j])))
#             # print(alternatives)
#             # print(len(alternatives))

#             found_right = None
#             found_wrong = None

#     # POG
#     # Sometimes '(A) ERRADA' is matched twice
#     if (len(alternatives) == 10):
#         alternatives = alternatives[0:10:2]

#     # for alt in alternatives:

#     #     start = alt['start']
#     #     end = alt['end']

#     #     print(repr(''.join(question[start:end])))

#     # Alternatives positions from reference of last question
#     return alternatives

# def evaluate_response(response, cot = []):

#     text = response["choices"][0]['text']
#     tokens = response["choices"][0]["logprobs"]['tokens']
#     logprobs = response["choices"][0]["logprobs"]['token_logprobs']

#     # print(text)
#     # print(repr(text))
#     # print(tokens)
#     # print()

#     pergunta = ['########', '####', '\n', '#', ' PER', 'G', 'UN', 'TA', ' #', '\n', '########', '####']
#     pergunta_pos = 0

#     for i in range(len(tokens)):

#         question = ''.join(tokens[i:i+len(pergunta)])
#         res = re.search(''.join(pergunta), question)

#         if (res != None):
#             pergunta_pos = i

#     question = tokens[pergunta_pos:]

#     # print(pergunta_pos)
#     # print(tokens[pergunta_pos:pergunta_pos+15])
#     # print(question)
#     # print("".join(question))

#     alternatives = get_alternatives_positions(question, pergunta_pos, cot)

#     # print(len(alternatives))

#     # for alt in alternatives:

#     #     start = alt['start']
#     #     end = alt['end']

#     #     print(repr(''.join(tokens[start:end])))
#     # #     print(''.join(tokens[start:end]))

#             # print()
#             # print("$$$$$$$$$$$$$$")
#             # print(tokens)
#             # print(last_question_pos)
#             # print(alternatives)
#             # print(logprobs)
#             # print(len(alternatives))
#             # print("$$$$$$$$$$$$$$")
#             # print()

#     alternatives_logprobs = []
#     alternatives_evaluation = []

#     # print()

#     for alt in alternatives:

#         start = alt['start']
#         end = alt['end']

#         # if (tokens[end+1] == '<|endoftext|>'):
#         #     end += 1

#         alt_logprobs = list(map(float, logprobs[start:end]))
#         alt_n_toks = end - start
#         prob = math.exp( (math.fsum(alt_logprobs) / alt_n_toks) )

#         # print(repr(''.join(tokens[start:end])))
#         # print(logprobs[start:end])

#         # print(prob)
#         # print(logprobs[start:end])
#         # print()

#         if(cot):
#             # ['A', ' altern', 'at', 'iva', ' \(', '[ABCDE]{1}', '\)', ' est', 'á', ' COR', 'RE', 'TA]
#             #                                          ^
#             alternatives_logprobs.append( {'alternative': tokens[start + 5],
#                                           'text':''.join(tokens[start:end]),
#                                           'prob': prob,
#                                           'temp_sum': math.fsum(alt_logprobs),
#                                           'temp_alt_n_toks': alt_n_toks} )
#         else:

#             if ( tokens[start] == '(' ):
#                 # (C) ERRADA.
#                 _0sh_start = start + 1
#             else:
#                 # C) ERRADA.
#                 _0sh_start = start

#             alternatives_logprobs.append( {'alternative': tokens[_0sh_start],
#                                           'text':''.join(tokens[start:end]),
#                                           'prob': prob,
#                                           'temp_sum': math.fsum(alt_logprobs),
#                                           'temp_alt_n_toks': alt_n_toks} )

#         #alternatives_evaluation.append(alternatives_logprobs[-1])

#     model_right_answer = max(alternatives_logprobs, key=lambda x:x['prob'])

#     #print(model_right_answer)

#     # print(alternatives_evaluation)

#     return model_right_answer, alternatives_logprobs

# ###################
# # CHANGE COT HERE #
# ###################

# model_right_answer, alternatives_evaluation = evaluate_response(response)
# # model_right_answer, alternatives_evaluation = evaluate_response(response, cot)

# print(model_right_answer['alternative'])

# for alt in alternatives_evaluation:
#     print(alt)

In [ ]:
#@title # IMPORTANT DEBUG CELL 0 #

# toks = tokenizer(cmd, max_length = max_length, padding = True, truncation = True)

# len_cmd = len(cmd)
# len_tokens = len(tokenizer.decode(toks['input_ids']))-2

# print(len_cmd)
# print(len_tokens)

In [ ]:
#@title # IMPORTANT DEBUG CELL 1 #

# s = ['\n', 'A', ' altern', 'at', 'iva', ' (', 'E', ')', ' est', 'á', ' COR', 'RE', 'TA']
# string = ''.join(s)

# correta_toks = ['\n', 'A', ' altern', 'at', 'iva', ' \(', '[ABCDE]{1}', '\)', ' est', 'á', ' COR', 'RE', 'TA']
# correta_str = "".join(correta_toks)
# correta_len = len(correta_str)

# print(f'correta_len: {correta_len}')
# print(f"s: {len(string)}")

# right = ''.join(s[0:13])
# found_right = re.search(correta_str, right)

# print(repr(correta_str))
# print(repr(right))

# print(found_right)




# s = ['\n', 'A', ')', ' COR', 'RE', 'TA', '.']
# string = ''.join(s)

# correta_0sh = ['\n', '[ABCDE]{1}', '\)', ' COR', 'RE', 'TA', '.']
# correta_str = "".join(correta_0sh)
# correta_len = len(correta_str)

# print(f'correta_len: {correta_len}')
# print(f"s: {len(string)}")

# right = ''.join(s)
# found_right = re.search(correta_str, right)

# print(repr(correta_str))
# print(repr(right))

# print(found_right)

In [ ]:
#=========#
# OLD COT #
#=========#

# cot = []

# decide_after = 0

# if(decide_after):
#     cot1 = 'Cabeçalho:\nSegundo Aristóteles, “na cidade com o melhor conjunto de normas e naquela dotada de homens absolutamente justos, os cidadãos não devem viver uma vida de trabalho trivial ou de negócios — esses tipos de vida são desprezíveis e incompatíveis com as qualidades morais —, tampouco devem ser agricultores os aspirantes à cidadania, pois o lazer é indispensável ao desenvolvimento das qualidades morais e à prática das atividades políticas”. VAN ACKER, T. Grécia. A vida cotidiana na cidade-Estado. São Paulo: Atual, 1994.\n\nEnunciado:\nO trecho, retirado da obra Política, de Aristóteles, permite compreender que a cidadania\n\nAlternativas:\n(A) possui uma dimensão histórica que deve ser criticada, pois é condenável que os políticos de qualquer época fiquem entregues à ociosidade, enquanto o resto dos cidadãos tem de trabalhar.\n(B) era entendida como uma dignidade própria dos grupos sociais superiores, fruto de uma concepção política profundamente hierarquizada da sociedade.\n(C) estava vinculada, na Grécia Antiga, a uma percepção política democrática, que levava todos os habitantes da pólis a participarem da vida cívica.\n(D) tinha profundas conexões com a justiça, razão pela qual o tempo livre dos cidadãos deveria ser dedicado às atividades vinculadas aos tribunais.\n(E) vivida pelos atenienses era, de fato, restrita àqueles que se dedicavam à política e que tinham tempo para resolver os problemas da cidade.\n\nExplicação:\nA alternativa (A) faz um julgamento de valor de uma passagem histórica. Não responde o que é cidadania conforme o texto. Portanto, a alternativa (A) está está ERRADA.\nA alternativa (B) interpreta acertadamente a sentença estabelecida por Aristóteles, que vincula o direito à cidadania a um grupo seleto de indivíduos limitando o pleno gozo da cidadania restringindo a um seleto grupo de pessoas superiores. Portanto, a alternativa (B) está está CORRETA.\nA alternativa (C) interpreta equivocadamente que todos habitantes teriam o mesmo direito cívico. O cabeçalho descreve que agricultores ou negociantes não deveriam ter o mesmo direito ao lazer como os políticos superiores. Portanto, a alternativa (C) está ERRADA.\nA alternativa (D) interpreta equivocadamente o texto já que o cabeçalho não diz nada sobre tempo livre dos cidadãos e de que deveriam se dedicar aos tribunais. Portanto, a alternativa (D) está ERRADA.\nA alternativa (E) interpreta equivocadamente o texto dizendo que os que tinham direito a cidadania eram aqueles que tinham tempo para se dedicar a política. Mas isso não é mencionado no cabeçalho. Ao contrário o texto restringe a cidadania a um seleto grupo de pessoas superiores. Portanto, a alternativa (E) está ERRADA.'
#     cot2 = 'Cabeçalho:\nEstima-se que haja atualmente no mundo 40 milhões de pessoas infectadas pelo HIV (o vírus que causa a AIDS), sendo que as taxas de novas infecções continuam crescendo, principalmente na África, Ásia e Rússia. Nesse cenário de pandemia, uma vacina contra o HIV teria imenso impacto, pois salvaria milhões de vidas. Certamente seria um marco na história planetária e também uma esperança para as populações carentes de tratamento antiviral e de acompanhamento médico. TANURI, A.; FERREIRA JUNIOR, O. C. Vacina contra Aids: desafios e esperanças. Ciência Hoje (44) 26, 2009 (adaptado).\n\nEnunciado:\nConsiderando o texto, uma vacina eficiente contra o HIV deveria:\n\nAlternativas:\n(A) induzir a imunidade, para proteger o organismo da contaminação viral.\n(B) ser capaz de alterar o genoma do organismo portador, induzindo a síntese de enzimas protetoras.\n(C) produzir antígenos capazes de se ligarem ao vírus, impedindo que este entre nas células do organismo humano.\n(D) ser amplamente aplicada em animais, visto que esses são os principais transmissores do vírus para os seres humanos.\n(E) estimular a imunidade, minimizando a transmissão do vírus por gotículas de saliva.\n\nExplicação:\nA alternativa (A) interpreta acertadamente que precisamos induzir imunidade de um organismo para protegê-lo contra futura infecções. Portanto, a alternativa (A) está CORRETA.\nA alternativa (B) interpreta equivocadamente que é necessário fazer mudanças no genoma de um organismo. Portanto, a alternativa (B) está ERRADA.\nA alternativa (C) interpreta equivocadamente que antígenos se ligam a moléculas de anticorpos ou receptores de células e não ao vírus. Portanto, a alternativa (C) está ERRADA.\nA alternativa (D) interpreta equivocadamente que HIV só é transmitido de animais para seres humanos. Portanto, a alternativa (D) está ERRADA.\nA alternativa (E) interpretaq equivocadamente que HIV é transmitido por gotículas de saliva. Portanto, a alternativa (E) está ERRADA.'
#     cot3 = 'Cabeçalho:\nUrgência emocional. Se tudo é para ontem, se a vida engata uma primeira e sai em disparada, se não há mais tempo para paradas estratégicas, caímos fatalmente no vício de querer que os amores sejam igualmente resolvidos num átimo de segundo. Temos pressa para ouvir "eu te amo". Não vemos a hora de que fiquem estabelecidas as regras de convívio: somos namorados, ficantes, casados, amantes? Urgência emocional. Uma cilada. Associamos diversas palavras ao AMOR: paixão, romance, sexo, adrenalina, palpitação. Esquecemos, no entanto, da palavra que viabiliza esse sentimento: "paciência". Amor sem paciência não vinga. Amor não pode ser mastigado e engolido com emergência, com fome desesperada. É uma refeição que pode durar uma vida. MEDEIROS, M. Disponível em: http://porumavidasimples.blogspot.com.br. Acesso em: 20 ago. 2017 (adaptado).\n\nEnunciado:\nNesse texto de opinião, as marcas linguísticas revelam uma situação distensa e de pouca formalidade, o que se evidencia pelo(a)\n\nAlternativas:\n(A) impessoalização ao longo do texto, como em: "se não há mais tempo".\n(B) construção de uma atmosfera de urgência, em palavras como: "pressa".\n(C) repetição de uma determinada estrutura sintática, como em: "Se tudo é para ontem".\n(D) ênfase no emprego da hipérbole, como em: "uma refeição que pode durar uma vida".\n(E) emprego de metáforas, como em: "a vida engata uma primeira e sai em disparada".\n\nExplicação:\nA alternativa (A) interpreta erroneamente que a impessoalização não é uma marca de pouca formalidade. Aliás, na sentença apontada na alternativa, o uso do verbo haver seria uma marca de formalidade. Portanto, a alternativa (A) está ERRADA.\nA alternativa (B) interpreta erroneamente o uso da palavra pressa, pois embora que o texto até tenha criado uma atmosfera de urgência, o texto na verdade fala exatamente sobre a importância da paciência e não da pressa. Portanto, a alternativa (B) está ERRADA.\nA alternativa (C) interpreta erroneamente que a estrutura sintática é repetida sistematicamente ao longo do texto. Portanto, a alternativa (C) está ERRADA.\nA alternativa (D) interpreta erroneamente que o texto enfatiza essa figura de linguagem, pois, para afirmar isto, a figura de linguagem deveria aparecer mais vezes. Portanto a alternativa (D) está ERRADA.\nA alternativa (E) interpreta acertadamente que o texto possui comparações implícitas que se caracterizam como emprego de metáforas. Portanto, a alternativa (E) está CORRETA.'
# else:
#     cot1 = 'Cabeçalho:\nSegundo Aristóteles, “na cidade com o melhor conjunto de normas e naquela dotada de homens absolutamente justos, os cidadãos não devem viver uma vida de trabalho trivial ou de negócios — esses tipos de vida são desprezíveis e incompatíveis com as qualidades morais —, tampouco devem ser agricultores os aspirantes à cidadania, pois o lazer é indispensável ao desenvolvimento das qualidades morais e à prática das atividades políticas”. VAN ACKER, T. Grécia. A vida cotidiana na cidade-Estado. São Paulo: Atual, 1994.\n\nEnunciado:\nO trecho, retirado da obra Política, de Aristóteles, permite compreender que a cidadania\n\nAlternativas:\n(A) possui uma dimensão histórica que deve ser criticada, pois é condenável que os políticos de qualquer época fiquem entregues à ociosidade, enquanto o resto dos cidadãos tem de trabalhar.\n(B) era entendida como uma dignidade própria dos grupos sociais superiores, fruto de uma concepção política profundamente hierarquizada da sociedade.\n(C) estava vinculada, na Grécia Antiga, a uma percepção política democrática, que levava todos os habitantes da pólis a participarem da vida cívica.\n(D) tinha profundas conexões com a justiça, razão pela qual o tempo livre dos cidadãos deveria ser dedicado às atividades vinculadas aos tribunais.\n(E) vivida pelos atenienses era, de fato, restrita àqueles que se dedicavam à política e que tinham tempo para resolver os problemas da cidade.\n\nExplicação:\nA alternativa (A) está ERRADA porque faz um julgamento de valor de uma passagem histórica. Não responde o que é cidadania conforme o texto.\nA alternativa (B) está CORRETA porque interpreta acertadamente a sentença estabelecida por Aristóteles, que vincula o direito à cidadania a um grupo seleto de indivíduos limitando o pleno gozo da cidadania restringindo a um seleto grupo de pessoas superiores.\nA alternativa (C) está ERRADA porque interpreta equivocadamente que todos habitantes teriam o mesmo direito cívico. O cabeçalho descreve que agricultores ou negociantes não deveriam ter o mesmo direito ao lazer como os políticos superiores.\nA alternativa (D) está ERRADA porque interpreta equivocadamente o texto já que o cabeçalho não diz nada sobre tempo livre dos cidadãos e de que deveriam se dedicar aos tribunais.\nA alternativa (E) está ERRADA porque interpreta equivocadamente o texto dizendo que os que tinham direito a cidadania eram aqueles que tinham tempo para se dedicar a política. Mas isso não é mencionado no cabeçalho. Ao contrário o texto restringe a cidadania a um seleto grupo de pessoas superiores.'
#     cot2 = 'Cabeçalho:\nEstima-se que haja atualmente no mundo 40 milhões de pessoas infectadas pelo HIV (o vírus que causa a AIDS), sendo que as taxas de novas infecções continuam crescendo, principalmente na África, Ásia e Rússia. Nesse cenário de pandemia, uma vacina contra o HIV teria imenso impacto, pois salvaria milhões de vidas. Certamente seria um marco na história planetária e também uma esperança para as populações carentes de tratamento antiviral e de acompanhamento médico. TANURI, A.; FERREIRA JUNIOR, O. C. Vacina contra Aids: desafios e esperanças. Ciência Hoje (44) 26, 2009 (adaptado).\n\nEnunciado:\nConsiderando o texto, uma vacina eficiente contra o HIV deveria:\n\nAlternativas:\n(A) induzir a imunidade, para proteger o organismo da contaminação viral.\n(B) ser capaz de alterar o genoma do organismo portador, induzindo a síntese de enzimas protetoras.\n(C) produzir antígenos capazes de se ligarem ao vírus, impedindo que este entre nas células do organismo humano.\n(D) ser amplamente aplicada em animais, visto que esses são os principais transmissores do vírus para os seres humanos.\n(E) estimular a imunidade, minimizando a transmissão do vírus por gotículas de saliva.\n\nExplicação:\nA alternativa (A) está CORRETA porque precisamos induzir imunidade de um organismo para protegê-lo contra futura infecções.\nA alternativa (B) está ERRADA pois não é possível fazer mudanças no genoma de um organismo.\nA alternativa (C) está ERRADA pois antígenos não se ligam ao vírus.\nA alternativa (D) está ERRADA pois pois HIV também é transmitido em humanos.\nA alternativa (E) está ERRADA pois HIV não se transmite por gotículas de saliva.'
#     cot3 = 'Cabeçalho:\nUrgência emocional. Se tudo é para ontem, se a vida engata uma primeira e sai em disparada, se não há mais tempo para paradas estratégicas, caímos fatalmente no vício de querer que os amores sejam igualmente resolvidos num átimo de segundo. Temos pressa para ouvir "eu te amo". Não vemos a hora de que fiquem estabelecidas as regras de convívio: somos namorados, ficantes, casados, amantes? Urgência emocional. Uma cilada. Associamos diversas palavras ao AMOR: paixão, romance, sexo, adrenalina, palpitação. Esquecemos, no entanto, da palavra que viabiliza esse sentimento: "paciência". Amor sem paciência não vinga. Amor não pode ser mastigado e engolido com emergência, com fome desesperada. É uma refeição que pode durar uma vida. MEDEIROS, M. Disponível em: http://porumavidasimples.blogspot.com.br. Acesso em: 20 ago. 2017 (adaptado).\n\nEnunciado:\nNesse texto de opinião, as marcas linguísticas revelam uma situação distensa e de pouca formalidade, o que se evidencia pelo(a)\n\nAlternativas:\n(A) impessoalização ao longo do texto, como em: "se não há mais tempo".\n(B) construção de uma atmosfera de urgência, em palavras como: "pressa".\n(C) repetição de uma determinada estrutura sintática, como em: "Se tudo é para ontem".\n(D) ênfase no emprego da hipérbole, como em: "uma refeição que pode durar uma vida".\n(E) emprego de metáforas, como em: "a vida engata uma primeira e sai em disparada".\n\nExplicação:\nA alternativa (A) está ERRADA porque impessoalização não é uma marca de pouca formalidade. Aliás, na sentença apontada na alternativa, o uso do verbo haver seria uma marca de formalidade.\nA alternativa (B) está ERRADA porque o texto até criou uma atmosfera de urgência, embora tenha sido para criticá-la. Na verdade o texto fala exatamente sobre a importância da paciência e não da pressa.\nA alternativa (C) está ERRADA porque a estrutura sintática não é repetida sistematicamente ao longo do texto.\nA alternativa (D) está ERRADA porque, embora possua hipérboles, para afirmar que o texto enfatiza essa figura de linguagem, ela deveria aparecer mais vezes.\nA alternativa (E) está CORRETA porque o texto possui comparações implícitas que se caracterizam como metáforas. Logo o texto emprega metáforas.'

# cmd_cot = 'Abaixo está uma lista de exemplos extraídos da prova do ENEM. Com base no cabeçalho e enunciado, para cada uma das alternativas (A, B, C, D, E) decida se a pergunta está ERRADA ou CORRETA e dê uma explicação para cada alternativa. Apenas uma alternativa é CORRETA.'

###################
# 0SHOT/COT SETUP #
###################

decide_after = 1

#=======#
# 0SHOT #
#=======#

# MELHOR ?
# Explica só a 13, mas nao o resto
# cmd_0shot = "Com base no cabeçalho e enunciado, responda a pergunta de múltipla escolha abaixo. Resposta desejada: para cada alternativa (A, B, C, D, E), classifique como CORRETA ou ERRADA e dê uma explicação."

cmd_0shot = "Com base no cabeçalho e enunciado, responda a pergunta de múltipla escolha abaixo. Resposta desejada: classifique cada alternativa (A, B, C, D, E) como CORRETA ou ERRADA e dê uma explicação."

#=====#
# COT #
#=====#

# cmd_cot = "Com base no cabeçalho e enunciado, para cada uma das alternativas (A, B, C, D, E) decida se a pergunta está ERRADA ou CORRETA e dê uma explicação para cada alternativa. Apenas uma alternativa é CORRETA."
# cmd_cot = "Formule uma explicação em cadeia que permita responder à questão de múltipla escolha abaixo. Apenas uma alternativa é CORRETA. Formato desejado: para cada alternativa, escreva uma sentença que classifica a alternativa como CORRETA ou ERRADA e justifica a resposta."
# cmd_cot = "Formule uma explicação em cadeia que permita responder à questão de múltipla escolha abaixo. Com base no cabeçalho e enunciado, para cada uma das alternativas (A, B, C, D, E) decida se a pergunta está ERRADA ou CORRETA e dê uma explicação para cada alternativa. Apenas uma alternativa é CORRETA."
cmd_cot = cmd_0shot

#########
# COT 1 #
#########

header1 = "Urgência emocional. Se tudo é para ontem, se a vida engata uma primeira e sai em disparada, se não há mais tempo para paradas estratégicas, caímos fatalmente no vício de querer que os amores sejam igualmente resolvidos num átimo de segundo. Temos pressa para ouvir \"eu te amo\". Não vemos a hora de que fiquem estabelecidas as regras de convívio: somos namorados, ficantes, casados, amantes? Urgência emocional. Uma cilada. Associamos diversas palavras ao AMOR: paixão, romance, sexo, adrenalina, palpitação. Esquecemos, no entanto, da palavra que viabiliza esse sentimento: \"paciência\". Amor sem paciência não vinga. Amor não pode ser mastigado e engolido com emergência, com fome desesperada. É uma refeição que pode durar uma vida. MEDEIROS, M. Disponível em: http://porumavidasimples.blogspot.com.br. Acesso em: 20 ago. 2017 (adaptado)."
statem1 = "Nesse texto de opinião, as marcas linguísticas revelam uma situação distensa e de pouca formalidade, o que se evidencia pelo(a)"
altA = "impessoalização ao longo do texto, como em: \"se não há mais tempo\"."
altB = "construção de uma atmosfera de urgência, em palavras como: \"pressa\"."
altC = "repetição de uma determinada estrutura sintática, como em: \"Se tudo é para ontem\"."
altD = "ênfase no emprego da hipérbole, como em: \"uma refeição que pode durar uma vida\"."
altE = "emprego de metáforas, como em: \"a vida engata uma primeira e sai em disparada\"."

question1 = "Cabeçalho:\n" + header1 + "\n\n" + "Enunciado:\n" + statem1 + "\n\n" + "Alternativas:\n" + "(A) " + altA + "\n(B) " + altB + "\n(C) " + altC+ "\n(D) " + altD+ "\n(E) " + altE

if(decide_after):
    expA = "A alternativa (A) interpreta erroneamente que a impessoalização não é uma marca de pouca formalidade, inclusive o uso do verbo haver representa uma marca de formalidade. Portanto, a alternativa (A) está ERRADA."
    expB = "A alternativa (B) interpreta erroneamente que o texto até criou uma atmosfera de urgência, embora tenha sido para criticá-la, e discute exatamente a importância da paciência e não da pressa. Portanto, a alternativa (B) está ERRADA."
    expC = "A alternativa (C) interpreta erroneamente que a estrutura sintática não é repetida sistematicamente ao longo do texto. Portanto, a alternativa (C) está ERRADA."
    expD = "A alternativa (D) interpreta erroneamente que, embora o texto possua hipérboles, para afirmar que a figura de linguagem é enfatizada, ela deveria aparecer mais vezes. Portanto, a alternativa (D) está ERRADA."
    expE = "A alternativa (E) interpreta corretamente que o texto possui comparações implícitas que se caracterizam como metáforas. Logo o texto emprega metáforas. Portanto, a alternativa (E) está CORRETA."
else:
    expA = "A alternativa (A) está ERRADA porque impessoalização não é uma marca de pouca formalidade, inclusive o uso do verbo haver representa uma marca de formalidade."
    expB = "A alternativa (B) está ERRADA porque o texto até criou uma atmosfera de urgência, embora tenha sido para criticá-la, e discute exatamente a importância da paciência e não da pressa."
    expC = "A alternativa (C) está ERRADA porque a estrutura sintática não é repetida sistematicamente ao longo do texto."
    expD = "A alternativa (D) está ERRADA porque, embora o texto possua hipérboles, para afirmar que a figura de linguagem é enfatizada, ela deveria aparecer mais vezes."
    expE = "A alternativa (E) está CORRETA porque o texto possui comparações implícitas que se caracterizam como metáforas. Logo o texto emprega metáforas."

cot1 = question1 + "\n\nExplicação:\n" + expA + "\n" + expB + "\n" + expC + "\n" + expD + "\n" + expE

#########
# COT 2 #
#########

header2 = "Sempre que a relevância do discurso entra em jogo, a questão torna-se política por definição, pois é o discurso que faz do homem um ser político. E tudo que os homens fazem, sabem ou experimentam só tem sentido na medida em que pode ser discutido. Haverá, talvez, verdades que ficam além da linguagem e que podem ser de grande relevância para o homem no singular, isto é, para o homem que, seja o que for, não é um ser político. Mas homens no plural, isto é, os homens que vivem e se movem e agem neste mundo, só podem experimentar o significado das coisas por poderem falar e ser inteligíveis entre si e consigo mesmos. ARENDT, H. A condição humana. Rio de Janeiro: Forense Universitária, 2004."
statem2 = "No trecho, a filósofa Hannah Arendt mostra a importância da linguagem no processo de"
altA = "entendimento da cultura."
altB = "aumento da criatividade."
altC = "percepção da individualidade."
altD = "melhoria da técnica."
altE = "construção da sociabilidade."

question2 = "Cabeçalho:\n" + header2 + "\n\n" + "Enunciado:\n" + statem2 + "\n\n" + "Alternativas:\n" + "(A) " + altA + "\n(B) " + altB + "\n(C) " + altC+ "\n(D) " + altD+ "\n(E) " + altE

if(decide_after):
    expA = "A alternativa (A) interpreta erroneamente que Hannah Arendt não trata do entendimento da cultura, mas da relação social entre as pessoas dessa cultura. Portanto, a alternativa (A) está ERRADA."
    expB = "A alternativa (B) interpreta erroneamente que Hannah Arendt não fala sobre criatividade, mas sobre a construção de laços entre as pessoas. Portanto, a alternativa (B) está ERRADA."
    expC = "A alternativa (C) interpreta erroneamente que a linguagem é utilizada no oposto da individualidade, em algo mais coletivo e social. Portanto, a alternativa (C) está ERRADA."
    expD = "A alternativa (D) interpreta erroneamente que o texto não fala de técnica, mas de laços. Portanto, a alternativa (D) está ERRADA."
    expE = "A alternativa (E) interpreta corretamente que a nossa sociabilidade se constrói a partir da linguagem, o que faz de nós seres políticos, no sentido de viver em sociedade, em ambientes coletivos. Portanto, a alternativa (E) está CORRETA."
else:
    expA = "A alternativa (A) está ERRADA porque Hannah Arendt não trata do entendimento da cultura, mas da relação social entre as pessoas dessa cultura."
    expB = "A alternativa (B) está ERRADA porque Hannah Arendt não fala sobre criatividade, mas sobre a construção de laços entre as pessoas."
    expC = "A alternativa (C) está ERRADA porque a linguagem é utilizada no oposto da individualidade, em algo mais coletivo e social."
    expD = "A alternativa (D) está ERRADA porque o texto não fala de técnica, mas de laços."
    expE = "A alternativa (E) está CORRETA porque a nossa sociabilidade se constrói a partir da linguagem, o que faz de nós seres políticos, no sentido de viver em sociedade, em ambientes coletivos."

cot2 = question2 + "\n\nExplicação:\n" + expA + "\n" + expB + "\n" + expC + "\n" + expD + "\n" + expE

#######
# COT #
#######

cot = []
cot.append(cot1)
cot.append(cot2)

In [ ]:
#######################
# QUERY ENEM QUESTION #
#######################

# PERTUNTA 11 (E) # \o/
# - 0shot sem explicaçao erra (C)
# - 0shot com explicação acerta (E)
# - few-shot-cot acerta (E) (mesmo tendo apontado duas CORRETAS)

# PERGUNTA 13 (E) #
# - 0shot sem explicaçao (D)
# - 0shot com explicação erra (V = λ . f ?)
# - few-shot-cot erra (até acerta no texto gerado, mas a alternativa perde nos logprobs)

# PERGUNTA 16 (D) # \o/
# - 0shot sem explicaçao erra (B)
# - 0shot com explicação erra (E)
# - few-shot-cot acerta (D)

# PERGUNTA 17 (B) #
# - 0shot sem explicaçao erra (A)
# - 0shot com explicação erra (D)
# - few-shot-cot erra (D) (mas a certeza aumenta e a difença entre a B diminui)

# PERGUNTA 55 (E) #
# - 0shot sem explicaçao erra (A)
# - 0shot com explicação erra (D)
# - few-shot-cot erra (C)

# PERGUNTA 89 (B) #
# - 0shot sem explicaçao erra (C)
# - 0shot com explicação erra (D)
# - few-shot-cot erra (E) (mas a certeza aumenta e a difença entre a B diminui)

# PERGUNTA 91 (B) #
# - 0shot sem explicaçao erra (E)
# - 0shot com explicação erra ()
# - few-shot-cot erra ()

question = 89
# max_length = 1024

############
# QUESTION #
############

ENEM.print_question(question)
#print()
#print(f"ENEM right answer: {ENEM.get_right_alternative(question)[0]}")
#print()

##########
# 0 SHOT #
##########

print("\n###############\n# 0-shot test #\n###############")

parsed_question, openai_response, model_right_answer, alternatives_evaluation = OPENAI.ask_ENEM_question(question, cmd_0shot)

print()
print(f"0shot answer: {model_right_answer['alternative']}")
print(f"0shot prob: {model_right_answer['prob']}")
print(f"0shot text: {repr(model_right_answer['text'])}")
print()
for alt in alternatives_evaluation:
    print(alt)

_0shot_answer = model_right_answer['alternative']

#################
# FEW SHOTS COT #
#################

print("\n#######################\n# (few/cot)-shot test #\n#######################")

parsed_question, openai_response, model_right_answer, alternatives_evaluation = OPENAI.ask_ENEM_question(question, cmd_cot, cot)

print()
print(f"cot-few-shot answer: {model_right_answer['alternative']}")
print(f"cot-few-shot prob: {model_right_answer['prob']}")
print(f"cot-few-shot text: {repr(model_right_answer['text'])}")
print()
for alt in alternatives_evaluation:
    print(alt)

few_cot_answer = model_right_answer['alternative']

###########
# RESULTS #
###########

print("\n###########\n# RESULTS #\n###########\n")

right_answer = ENEM.get_right_alternative(question)[0]

if (_0shot_answer == right_answer):
    print(f"{bcolors.OKGREEN}0-shot explicado ACERTOU a resposta{bcolors.ENDC}")
else:
    print(f"{bcolors.FAIL}0-shot explicado ERROU a resposta{bcolors.ENDC}")

if (few_cot_answer == right_answer):
    print(f"{bcolors.OKGREEN}few-shot-cot explicado ACERTOU a resposta{bcolors.ENDC}")
else:
    print(f"{bcolors.FAIL}few-shot-cot explicado ERROU a resposta{bcolors.ENDC}")


##############
# Questão 89 #
##############

Ano/Versão: 2010-1

Cabeçalho:

Antes, eram apenas as grandes cidades que se apresentavam como o império da técnica, objeto de
modificações, suspensões, acréscimos, cada vez mais sofisticadas e carregadas de artifício. Esse mundo artificial inclui,
hoje, o mundo rural. SANTOS, M. A Natureza do Espaço. São Paulo: Hucitec, 1996. 

Enunciado:

Considerando a transformação mencionada no texto, uma consequência socioespacial que caracteriza o atual mundo rural brasileiro
é 

Alternativas:
 A  - a redução do processo de concentração de terras.
[B] - o aumento do aproveitamento de solos menos férteis.
 C  - a ampliação do isolamento do espaço rural.
 D  - a estagnação da fronteira agrícola do país.
 E  - a diminuição do nível de emprego formal.

###############
# 0-shot test #
###############

0shot answer: C
0shot prob: 0.8775152886746171
0shot text: 'C) CORRETA - A ampliação do isolamento do espaço rural é uma consequência socioespacial que car

In [ ]:
#@title CHECK DATA FROM PLAYGROUND

##############################
# CHECK DATA FROM PLAYGROUND #
##############################

# ENEM.print_question(6)
# ENEM.print_question(88)

# print()
# print(f"A: {math.exp( -19.50 / 18 )}")
# print(f"B: {math.exp( -21.99 / 23 )}")
# print(f"C: {math.exp( -19.81 / 18 )}")
# print(f"D: {math.exp( -22.02 / 20 )}")
# print(f"E: {math.exp( -22.11 / 17 )}")

#################
# PRINT RESULTS #
#################

# ENEM.print_question(11)
# print(parsed_question)
# print(openai_response["choices"][0]['text'])
# print(openai_response)

# tokens = openai_response["choices"][0]["logprobs"]['tokens']

tokens = ['Com', ' base', ' no', ' c', 'abe', 'ç', 'al', 'ho', ' e', ' en', 'unci', 'ado', ',', ' para', ' c', 'ada', ' u', 'ma', ' d', 'as', ' altern', 'at', 'ivas', ' (', 'A', ',', ' B', ',', ' C', ',', ' D', ',', ' E', ')', ' dec', 'ida', ' se', ' a', ' per', 'g', 'unta', ' est', 'á', ' ER', 'R', 'ADA', ' o', 'u', ' COR', 'RE', 'TA', ' e', ' d', 'ê', ' u', 'ma', ' expl', 'ica', 'ç', 'ão', ' para', ' c', 'ada', ' altern', 'at', 'iva', '.', ' A', 'pen', 'as', ' u', 'ma', ' altern', 'at', 'iva', ' é', ' COR', 'RE', 'TA', '.', '\n', '\n', '########', '####', '\n', '#', ' PER', 'G', 'UN', 'TA', ' #', '\n', '########', '####', '\n', '\n', 'C', 'abe', 'ç', 'al', 'ho', ':', '\n', 'O', ' prog', 'resso', ' da', ' te', 'cn', 'olog', 'ia', ' introdu', 'z', 'iu', ' divers', 'os', ' arte', 'fat', 'os', ' ger', 'ad', 'ores', ' de', ' camp', 'os', ' e', 'let', 'rom', 'agn', 'é', 'tic', 'os', '.', ' U', 'ma', ' d', 'as', ' m', 'ais', ' em', 'p', 'reg', 'adas', ' in', 'ven', 'ç', 'bytes:\\xc3', 'bytes:\\xb5', 'es', ' n', 'essa', 'bytes: \\xc3', 'bytes:\\xa1', 'rea', ' s', 'ão', ' os', ' tele', 'f', 'ones', ' cel', 'ul', 'ares', ' e', ' smartphones', '.', ' As', ' te', 'cn', 'olog', 'ias', ' de', ' trans', 'miss', 'ão', ' de', ' cel', 'ular', ' at', 'ual', 'ment', 'e', ' em', ' us', 'o', ' no', ' Bras', 'il', ' contempl', 'am', ' do', 'is', ' s', 'ist', 'em', 'as', '.', ' O', ' prime', 'iro', ' de', 'les', ' é', ' oper', 'ado', ' ent', 're', ' as', ' frequ', 'ê', 'n', 'ci', 'as', ' de', ' 800', ' MHz', ' e', ' 900', ' MHz', ' e', ' constit', 'ui', ' os', ' ch', 'am', 'ados', ' s', 'ist', 'em', 'as', ' TD', 'MA', '/', 'CD', 'MA', '.', ' J', 'á', ' a', ' te', 'cn', 'olog', 'ia', ' G', 'SM', ',', ' o', 'cup', 'a', ' a', ' frequ', 'ê', 'nc', 'ia', ' de', ' 1', '.', '800', ' MHz', '.', '\n', '\n', 'En', 'unci', 'ado', ':', '\n', 'Consider', 'ando', ' que', ' a', ' intens', 'id', 'ade', ' de', ' trans', 'miss', 'ão', ' e', ' o', ' n', 'í', 'vel', ' de', ' rece', 'p', 'ç', 'ão', 'bytes: \\xe2\\x80', 'bytes:\\x9c', 'cel', 'ular', 'bytes:\\xe2\\x80', 'bytes:\\x9d', ' se', 'jam', ' os', ' mes', 'mos', ' para', ' as', ' te', 'cn', 'olog', 'ias', ' de', ' trans', 'miss', 'ão', ' TD', 'MA', '/', 'CD', 'MA', ' o', 'u', ' G', 'SM', ',', ' se', ' um', ' eng', 'en', 'he', 'iro', ' t', 'iver', ' de', ' esc', 'ol', 'her', ' ent', 're', ' as', ' du', 'as', ' te', 'cn', 'olog', 'ias', ' para', ' ob', 'ter', ' a', ' mes', 'ma', ' co', 'bert', 'ura', ',', ' lev', 'ando', ' em', ' consider', 'a', 'ç', 'ão', ' ap', 'en', 'as', ' o', ' n', 'ú', 'mer', 'o', ' de', ' anten', 'as', ' em', ' u', 'ma', ' reg', 'i', 'ão', ',', ' ele', ' de', 'ver', 'á', ' esc', 'ol', 'her', ':', '\n', '\n', 'Altern', 'at', 'ivas', ':', '\n', '(', 'A', ')', ' a', ' te', 'cn', 'olog', 'ia', ' G', 'SM', ',', ' po', 'is', ' é', ' a', ' que', ' opera', ' com', ' on', 'd', 'as', ' de', ' ma', 'ior', ' comp', 'rim', 'ento', ' de', ' on', 'da', '.', '\n', '(', 'B', ')', ' a', ' te', 'cn', 'olog', 'ia', ' TD', 'MA', '/', 'CD', 'MA', ',', ' po', 'is', ' é', ' a', ' que', ' ap', 'resent', 'a', ' E', 'fe', 'ito', ' Do', 'pp', 'ler', ' m', 'ais', ' pron', 'unci', 'ado', '.', '\n', '(', 'C', ')', ' a', ' te', 'cn', 'olog', 'ia', ' G', 'SM', ',', ' po', 'is', ' é', ' a', ' que', ' util', 'iza', ' on', 'd', 'as', ' que', ' se', ' propag', 'am', ' com', ' ma', 'ior', ' vel', 'oc', 'id', 'ade', '.', '\n', '(', 'D', ')', ' qual', 'quer', ' u', 'ma', ' d', 'as', ' du', 'as', ',', ' po', 'is', ' as', ' d', 'if', 'eren', 'ç', 'as', ' nas', ' frequ', 'ê', 'n', 'ci', 'as', ' s', 'ão', ' compens', 'adas', ' pel', 'as', ' d', 'if', 'eren', 'ç', 'as', ' nos', ' comp', 'rim', 'ent', 'os', ' de', ' on', 'da', '.', '\n', '(', 'E', ')', ' qual', 'quer', ' u', 'ma', ' d', 'as', ' du', 'as', ',', ' po', 'is', ' n', 'esse', ' cas', 'o', ' as', ' intens', 'id', 'ades', ' dec', 'a', 'em', ' ig', 'ual', 'ment', 'e', ' da', ' mes', 'ma', ' form', 'a', ',', ' independent', 'ement', 'e', ' da', ' frequ', 'ê', 'nc', 'ia', '.', '\n', '\n', 'Expl', 'ica', 'ç', 'ão', ':', '\n', 'A', ')', ' A', ' te', 'cn', 'olog', 'ia', ' G', 'SM', ' opera', ' com', ' on', 'd', 'as', ' de', ' ma', 'ior', ' comp', 'rim', 'ento', ' de', ' on', 'da', ',', ' mas', ' is', 'so', ' n', 'ão', ' signific', 'a', ' que', ' el', 'a', ' ten', 'ha', ' u', 'ma', ' co', 'bert', 'ura', ' mel', 'hor', '.', '\n', '\n', 'B', ')', ' A', ' te', 'cn', 'olog', 'ia', ' TD', 'MA', '/', 'CD', 'MA', ' ap', 'resent', 'a', ' E', 'fe', 'ito', ' Do', 'pp', 'ler', ' m', 'ais', ' pron', 'unci', 'ado', ',', ' mas', ' is', 'so', ' n', 'ão', ' signific', 'a', ' que', ' el', 'a', ' ten', 'ha', ' u', 'ma', ' co', 'bert', 'ura', ' mel', 'hor', '.', '\n', '\n', 'C', ')', ' A', ' te', 'cn', 'olog', 'ia', ' G', 'SM', ' util', 'iza', ' on', 'd', 'as', ' que', ' se', ' propag', 'am', ' com', ' ma', 'ior', ' vel', 'oc', 'id', 'ade', ',', ' mas', ' is', 'so', ' n', 'ão', ' signific', 'a', ' que', ' el', 'a', ' ten', 'ha', ' u', 'ma', ' co', 'bert', 'ura', ' mel', 'hor', '.', '\n', '\n', 'D', ')', ' Qual', 'quer', ' u', 'ma', ' d', 'as', ' du', 'as', ' te', 'cn', 'olog', 'ias', ' p', 'ode', ' ser', ' esc', 'ol', 'h', 'ida', ',', ' po', 'is', ' as', ' d', 'if', 'eren', 'ç', 'as', ' nas', ' frequ', 'ê', 'n', 'ci', 'as', ' s', 'ão', ' compens', 'adas', ' pel', 'as', ' d', 'if', 'eren', 'ç', 'as', ' nos', ' comp', 'rim', 'ent', 'os', ' de', ' on', 'da', '.', '\n', '\n', 'E', ')', ' Qual', 'quer', ' u', 'ma', ' d', 'as', ' du', 'as', ' te', 'cn', 'olog', 'ias', ' p', 'ode', ' ser', ' esc', 'ol', 'h', 'ida', ',', ' po', 'is', ' n', 'esse', ' cas', 'o', ' as', ' intens', 'id', 'ades', ' dec', 'a', 'em', ' ig', 'ual', 'ment', 'e', ' da', ' mes', 'ma', ' form', 'a', ',', ' independent', 'ement', 'e', ' da', ' frequ', 'ê', 'nc', 'ia', '.']

# davinci-002 sometimes does not close the string at the end
last_token = tokens[(len(tokens)-1)]
if( last_token != '\n' or last_token != '<|endoftext|>'):
    tokens.append('<|endoftext|>')

line_breaks = []
for i, tok in enumerate(tokens):
    res = re.search('\n|<|endoftext|>', tok)
    if (res != None):
        line_breaks.append(i)

for i in range(len(line_breaks)-1):    
    string = "".join(tokens[line_breaks[i]+1:line_breaks[i+1]])
    print(string)

# alternatives = []

# # # In multiline mode, ^ matches the position immediately following a newline
# # # and $ matches the position immediately preceding a newline.
# regex = ".*(CORRETA|ERRADA).*"

# for i in range(len(line_breaks)-1):
    
#     string = "".join(tokens[line_breaks[i]+1:line_breaks[i+1]])

#     found_alternative = re.search(regex, string)#, re.MULTILINE)

#     if (found_alternative != None):
#       alternatives.append( { 'start': (line_breaks[i]+1), 'end': (line_breaks[i+1]) } )
#       print(string)


############
# PERGUNTA #
############

Cabeçalho:
O progresso da tecnologia introduziu diversos artefatos geradores de campos eletromagnéticos. Uma das mais empregadas invençbytes:\xc3bytes:\xb5es nessabytes: \xc3bytes:\xa1rea são os telefones celulares e smartphones. As tecnologias de transmissão de celular atualmente em uso no Brasil contemplam dois sistemas. O primeiro deles é operado entre as frequências de 800 MHz e 900 MHz e constitui os chamados sistemas TDMA/CDMA. Já a tecnologia GSM, ocupa a frequência de 1.800 MHz.

Enunciado:
Considerando que a intensidade de transmissão e o nível de recepçãobytes: \xe2\x80bytes:\x9ccelularbytes:\xe2\x80bytes:\x9d sejam os mesmos para as tecnologias de transmissão TDMA/CDMA ou GSM, se um engenheiro tiver de escolher entre as duas tecnologias para obter a mesma cobertura, levando em consideração apenas o número de antenas em uma região, ele deverá escolher:

Alternativas:
(A) a tecnologia GSM, pois é a que opera com ondas de maior comprimen

In [ ]:
#@title TODO [broken]: Save data per question

########################
# REAL TESTS FEW SHOTS #
########################

# path = "/content/drive/My Drive/doc_usp/IA024/artigo/" + OPENAI.model + "-"

# def query_model(OPENAI, cmd_0shot, cmd_cot, cot, range):

#     right_answers = []
#     question_year = []
#     ___0shot_model_replies = []
#     few_shot_model_replies = []

#     for question in tqdm(range):

#         right_answers.append(ENEM.get_right_alternative(question)[0])
#         question_year.append(ENEM.get_question_year(question))

#         #========#
#         # 0 shot #
#         #========#

#         parsed_question, openai_response, model_right_answer, alternatives_evaluation = OPENAI.ask_ENEM_question(question, cmd_0shot)
#         reply = {'parsed_question': parsed_question,
#                   'openai_response': openai_response,
#                   'model_right_answer': model_right_answer,
#                   'alternatives_evaluation': alternatives_evaluation}  
#         ___0shot_model_replies.append(reply)

#         #====================#
#         # SAVING each 0 shot #
#         #====================#

#         full_path = path + "___0shot_bytearray_answers" + ".data"
#         json_object = json.dumps(___0shot_model_replies[-1])
#         with open(full_path, "a") as outfile:
#             outfile.write(json_object)
#         outfile.close()

#         full_path = path + "___0shot_answers" + ".csv"
#         d = {'question_year': [question_year[-1]], 'model_answers': [reply['model_right_answer']['alternative']], 'right_answers': [right_answers[-1]]}
#         df = pd.DataFrame(d)
#         with open(full_path, 'a') as f:
#           if (question != 0):
#             df.to_csv(f, header=False)
#           else:
#             df.to_csv(f)

#         f.close()

#         #========#
#         # f shot #
#         #========#

#         parsed_question, openai_response, model_right_answer, alternatives_evaluation = OPENAI.ask_ENEM_question(question, cmd_cot, cot)
#         reply = {'parsed_question': parsed_question,
#                   'openai_response': openai_response,
#                   'model_right_answer': model_right_answer,
#                   'alternatives_evaluation': alternatives_evaluation}  
#         few_shot_model_replies.append(reply)

#         #======================#
#         # SAVING each few shot #
#         #======================#

#         full_path = path + "few_shot_bytearray_answers" + ".data"
#         json_object = json.dumps(few_shot_model_replies[-1])
#         with open(full_path, "a") as outfile:
#             outfile.write(json_object)
#         outfile.close()

#         full_path = path + "few_shot_answers" + ".csv"
#         d = {'question_year': [question_year[-1]], 'model_answers': [reply['model_right_answer']['alternative']], 'right_answers': [right_answers[-1]]}
#         df = pd.DataFrame(d)
#         with open(full_path, 'a') as f:
#           if (question != 0):
#             df.to_csv(f, header=False)
#           else:
#             df.to_csv(f)

#         f.close()

#         # Follow the free request rate limit on OpenAI
#         time.sleep(10)

#     ___0shot_answers = [d['model_right_answer']['alternative'] for d in ___0shot_model_replies]
#     few_shot_answers = [d['model_right_answer']['alternative'] for d in few_shot_model_replies]

#     ___0shot_accuracy = sum(x == y for x, y in zip(___0shot_answers, right_answers))
#     few_shot_accuracy = sum(x == y for x, y in zip(few_shot_answers, right_answers))

#     return ___0shot_model_replies, ___0shot_answers, ___0shot_accuracy, few_shot_model_replies, few_shot_answers, few_shot_accuracy, right_answers

In [ ]:
########################
# REAL TESTS FEW SHOTS #
########################

path = "/content/drive/My Drive/doc_usp/IA024/artigo/" + OPENAI.model + "-"

def query_model(OPENAI, cmd_0shot, cmd_cot, cot, range):

    right_answers = []
    question_year = []
    ___0shot_model_replies = []
    few_shot_model_replies = []

    for question in tqdm(range):

        right_answers.append(ENEM.get_right_alternative(question)[0])
        question_year.append(ENEM.get_question_year(question))

        #========#
        # 0 shot #
        #========#

        parsed_question, openai_response, model_right_answer, alternatives_evaluation = OPENAI.ask_ENEM_question(question, cmd_0shot)
        reply = {'parsed_question': parsed_question,
                  'openai_response': openai_response,
                  'model_right_answer': model_right_answer,
                  'alternatives_evaluation': alternatives_evaluation}  
        ___0shot_model_replies.append(reply)

        #========#
        # f shot #
        #========#

        parsed_question, openai_response, model_right_answer, alternatives_evaluation = OPENAI.ask_ENEM_question(question, cmd_cot, cot)
        reply = {'parsed_question': parsed_question,
                  'openai_response': openai_response,
                  'model_right_answer': model_right_answer,
                  'alternatives_evaluation': alternatives_evaluation}  
        few_shot_model_replies.append(reply)

        # Follow the free request rate limit on OpenAI
        time.sleep(10)

    ___0shot_answers = [d['model_right_answer']['alternative'] for d in ___0shot_model_replies]
    few_shot_answers = [d['model_right_answer']['alternative'] for d in few_shot_model_replies]

    ___0shot_accuracy = sum(x == y for x, y in zip(___0shot_answers, right_answers))
    few_shot_accuracy = sum(x == y for x, y in zip(few_shot_answers, right_answers))

    #===================#
    # SAVING all 0 shot #
    #===================#

    full_path = path + "___0shot_bytearray_answers" + ".data"
    json_object = json.dumps(___0shot_model_replies)
    with open(full_path, "w") as outfile:
        outfile.write(json_object)
    outfile.close()

    full_path = path + "___0shot_answers" + ".csv"
    d = {'question_year': question_year, 'model_answers': ___0shot_answers, 'right_answers': right_answers}
    df = pd.DataFrame(d)
    with open(full_path, 'w') as f:
      df.to_csv(f)
    f.close()

    #=====================#
    # SAVING all few shot #
    #=====================#

    full_path = path + "few_shot_bytearray_answers" + ".data"
    json_object = json.dumps(few_shot_model_replies)
    with open(full_path, "w") as outfile:
        outfile.write(json_object)
    outfile.close()

    full_path = path + "few_shot_answers" + ".csv"
    d = {'question_year': question_year, 'model_answers': few_shot_answers, 'right_answers': right_answers}
    df = pd.DataFrame(d)
    with open(full_path, 'w') as f:
      df.to_csv(f)
    f.close()

    return ___0shot_model_replies, ___0shot_answers, ___0shot_accuracy, few_shot_model_replies, few_shot_answers, few_shot_accuracy, right_answers

In [ ]:
#######################
# FEW SHOW EVALUATION #
#######################

print()
print("###############################")
print("# GPT-3.5 ACCURACY EVALUATION #")
print("###############################")
print()
print("Evaluation of " + OPENAI.model)
print()

# EXAME TESTE
# questions_to_ask = range(0,5)

# EXAME 2009
questions_to_ask = range(0,89)

# EXAME TOTAL
# questions_to_ask = range(0,len(ENEM))

___0shot_model_replies, ___0shot_answers, ___0shot_accuracy, few_shot_model_replies, few_shot_answers, few_shot_accuracy, right_answers = query_model(OPENAI, cmd_0shot, cmd_cot, cot, questions_to_ask)
# ___0shot_model_answers, few_shot_model_answers, right_answers = query_model(OPENAI, cmd_0shot, cmd_cot, cot, questions_to_ask)

#========#
# 0 shot #
#========#

incorrect_0shot = [{'question': q, 'model_answer': i, 'right_answer': j} for i, j, q in
                   zip(___0shot_answers, right_answers, range(len(right_answers))) if i != j]

print()
print("Accuracy - 0shot: {0:.2f} %".format(___0shot_accuracy/len(questions_to_ask) * 100))

#========#
# f shot #
#========#

incorrect_fshot = [{'question': q, 'model_answer': i, 'right_answer': j} for i, j, q in
                   zip(few_shot_answers, right_answers, range(len(right_answers))) if i != j]

print("Accuracy - fshot: {0:.2f} %".format(few_shot_accuracy/len(questions_to_ask) * 100))
print()

print("############################")
print("# 0 shot wrong predictions #")
print("############################")
print()
print("Wrong Predictions - 0shot:\n")
print(*incorrect_0shot, sep='\n')
print()

print("############################")
print("# f shot wrong predictions #")
print("############################")
print()
print("Wrong Predictions - fshot:\n")
print(*incorrect_fshot, sep='\n')
print()

###############
# ALL ANSWERS #
###############

# ___0shot = [{'question': q, 'model_answer': i, 'right_answer': j} for i, j, q in
#            zip(___0shot_answers, right_answers, range(len(right_answers)))]

# print("######################")
# print("# 0 shot all answers #")
# print("######################")
# print()
# print("All Predictions - 0shot:\n")
# print(*___0shot, sep='\n')
# print()

# few_shot = [{'question': q, 'model_answer': i, 'right_answer': j} for i, j, q in
#            zip(few_shot_answers, right_answers, range(len(right_answers)))]

# print("######################")
# print("# f shot all answers #")
# print("######################")
# print()
# print("All Predictions - fshot:\n")
# print(*few_shot, sep='\n')
# print()


###############################
# GPT-3.5 ACCURACY EVALUATION #
###############################

Evaluation of text-davinci-003



100%|██████████| 89/89 [52:18<00:00, 35.27s/it]



Accuracy - 0shot: 52.81 %
Accuracy - fshot: 68.54 %

############################
# 0 shot wrong predictions #
############################

Wrong Predictions - 0shot:

{'question': 1, 'model_answer': 'C', 'right_answer': 'A'}
{'question': 3, 'model_answer': 'C', 'right_answer': 'E'}
{'question': 8, 'model_answer': 'B', 'right_answer': 'C'}
{'question': 11, 'model_answer': 'C', 'right_answer': 'E'}
{'question': 12, 'model_answer': 'C', 'right_answer': 'D'}
{'question': 13, 'model_answer': 'B', 'right_answer': 'E'}
{'question': 15, 'model_answer': 'B', 'right_answer': 'C'}
{'question': 16, 'model_answer': 'B', 'right_answer': 'D'}
{'question': 17, 'model_answer': 'A', 'right_answer': 'B'}
{'question': 20, 'model_answer': 'C', 'right_answer': 'E'}
{'question': 21, 'model_answer': 'D', 'right_answer': 'A'}
{'question': 23, 'model_answer': 'B', 'right_answer': 'C'}
{'question': 25, 'model_answer': 'D', 'right_answer': 'A'}
{'question': 28, 'model_answer': 'D', 'right_answer': 'E'}
{'quest

In [ ]:
#==============#
# READING data #
#==============#

# path = "/content/drive/My Drive/doc_usp/IA024/artigo/" + OPENAI.model + "-"

# full_path = path + "___0shot_bytearray_answers" + ".data"
# # full_path = path + "few_shot_bytearray_answers" + ".data"

# with open(full_path, 'r') as openfile:
#     data = json.load(openfile)
 
# print(data)
# print()
# print(type(data))
# print()
# for d in data:
#   print(d['model_right_answer']['alternative'])

In [ ]:
#############
# STOP HERE #
#############

print(FIM)

In [ ]:
###########
# PART II #
###########

# Ricardo

### Saving embeddings from BERTimbau


In [ ]:
##############
# EMBEDDINGS #
##############

from transformers import BertModel
bert_pretrained = BertModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

def get_BERT_embeddings(ENEM_data, model, q_id):
    with torch.no_grad():
        reps_q = model( torch.tensor([ENEM_data.headers_ids[q_id]]),
                       attention_mask = torch.tensor([ENEM_data.headers_masks[q_id]]), 
                       output_hidden_states=True).last_hidden_state.squeeze(0).numpy()

        reps_s = model( torch.tensor([ENEM_data.statements_ids[q_id]]),
                       attention_mask = torch.tensor([ENEM_data.statements_masks[q_id]]), 
                       output_hidden_states=True).last_hidden_state.squeeze(0).numpy()    
        
        reps_o = model( torch.tensor(ENEM_data.answers[q_id][0]),
                       attention_mask = torch.tensor(ENEM_data.answers[q_id][1]), 
                       output_hidden_states=True).last_hidden_state.squeeze(0).numpy()            
    return(reps_q, reps_s, reps_o ) 


#embeddings_data = []
#for v, i in zip(ENEM.enem_version, ENEM.questions):
#  q, s, o = get_BERT_embeddings(ENEM_data, model= bert_pretrained, q_id = int(i['id']))
#  embeddings_data.append({'version':v, 'id': i['id'], 'q':q, 's':s, 'o':o})

#embeddings_data[0]

#import json
#with open('ebmeddings.json', 'w') as f:
#    json.dump(embeddings_data , f)

#import pickle
#with open('embeddings.pkl', 'wb') as f:
#    pickle.dump(embeddings_data, f)

   


#print(q.shape)
#print(s.shape)
#print(o.shape)


#text_question = pd.read_csv("/content/drive/MyDrive/unicamp - IA024 /projeto_enem/text_question.csv") 
#text_option = pd.read_csv("/content/drive/MyDrive/unicamp - IA024 /projeto_enem/text_option.csv") 

#reps.last_hidden_state[:, 0]
#reps.hidden_states[-1].shape
#torch.equal(reps.hidden_states[-1], reps.last_hidden_state)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


* Cosine similarity

In [ ]:
import pickle
with open('/content/drive/MyDrive/unicamp - IA024 /projeto_enem/embeddings.pkl', 'rb') as f:
    embeddings_data = pickle.load(f)

for i in range(3):
  print(ENEM.print_question(i))
  print()
  print("Question: " + embeddings_data[i]['id'] + " Dimensions of the header: " + str(embeddings_data[i]['q'].shape))
  print("Question: " + embeddings_data[i]['id'] + " Dimensions of the statment: " + str(embeddings_data[i]['s'].shape))
  print("Question: " + embeddings_data[i]['id'] + " Dimensions of options: " + str(embeddings_data[i]['o'].shape))
  
  print()
  print(ENEM_data.headers_ids[i])
  print()
  print(ENEM_data.statements_ids[i])
  print()
  print(ENEM_data.answers[i])


import matplotlib.pyplot as plt

plt.hist(embeddings_data[0]['q'].mean(axis=0), bins=20, color = "red")
plt.show()

plt.hist(embeddings_data[0]['q'][np.array(ENEM_data.headers_ids[0]) != 0].mean(axis=0), bins=20, color = "red")
plt.show()

embeddings_data[0]['q'].mean(axis=0) == embeddings_data[0]['q'][np.array(ENEM_data.headers_ids[0]) != 0].mean(axis=0)




#############
# Questão 0 #
#############

Ano/Versão: 2009-1

Cabeçalho:
 A  atmosfera  terrestre  é  composta  pelos  gases nitrogênio (N2) e oxigênio (O2), que somam cerca de 99%, e por gases traços, entre eles o gás carbônico (CO2), vapor de água (H2O), metano (CH4), ozônio (O3) e o óxido nitroso (N2O), que compõem o restante 1% do ar que respiramos. Os  gases  traços,  por  serem  constituídos  por  pelo  menos três  átomos,  conseguem  absorver  o  calor  irradiado  pela Terra, aquecendo o planeta. Esse fenômeno, que acontece há bilhões de anos, é chamado de efeito estufa. A partir da Revolução  Industrial  (século  XIX),  a  concentração  de gases  traços  na  atmosfera,  em  particular  o CO2,  tem aumentado significativamente, o que resultou no aumento da  temperatura  em  escala  global.  Mais  recentemente, outro fator tornou-se diretamente envolvido no aumento da concentração de CO2 na atmosfera: o desmatamento.  BROWN, I. F.; ALECHANDRE, A. S. Conceitos básicos sobre clim

In [ ]:
* Model 1: mean of embeddings

In [ ]:
embeddings_data2 = []

for i in ENEM.questions:
  q_id  = int(i['id'])
  q2 = embeddings_data[q_id]['q'][np.array(ENEM_data.headers_ids[q_id]) != 0].mean(axis=0)
  s2 = embeddings_data[q_id]['s'][np.array(ENEM_data.statements_ids[q_id]) != 0].mean(axis=0)
  answers = []
  for a in (embeddings_data[q_id]['o']):
    answers.append(np.array(a).mean(axis=0))
  
  embeddings_data2.append({'id': q_id, 'q2':q2, 's2':s2, 'o2':o2})

#embeddings_data[0]['q'][np.array(ENEM_data.headers_ids[0]) != 0].mean(axis=0)

In [ ]:

for i in ENEM.questions:
  print(i)

answers = []
for a in (embeddings_data[0]['o']):
   print(a.shape)
   print(a)
   print(np.array(a).mean(axis=0).shape)
  #o2 = a[0].mean(axis=1)
  #answers.append(o2)



### Fine tuning Bertimbau

#### Salvando a base em um dataframe

In [ ]:
data_enem0 = [ENEM.__getitem__(i) for i in range(len(ENEM))]

data_enem0[0]

data_enem =  pd.DataFrame(data_enem0, columns = ['enem_version', 'question.attrib', 'header', 'statement', 'answers'])  

def df_column_to_columns(df, column_name):
    df = pd.concat([df.drop([column_name], axis=1), df[column_name].apply(pd.Series)], axis=1)
    return df

data_enem = df_column_to_columns(data_enem, 'question.attrib')
data_enem = df_column_to_columns(data_enem, 'answers')
data_enem = df_column_to_columns(data_enem, 1)

data_enem.columns = ['enem_version', 'header', 'statement','CE','DS','EK','IC','MR','TC','id','image',
                 'gab', 'A','B','C','D','E']

def select_list_element(df, column_name, key, value):
    df[column_name] = df[column_name].apply(lambda x: [d for d in x if d[key] == value])
    return df   


data_enem = select_list_element(data_enem, 'gab', 'correct', 'Yes')


def get_value_from_dict(df, column_name, key):
    df[column_name] = df[column_name].apply(lambda x: x[0][key])
    return df

data_enem = get_value_from_dict(data_enem, 'gab', 'id')

data_enem.info()

ENEM.print_question(0)

data_enem.to_csv("/content/drive/MyDrive/unicamp - IA024 /projeto_enem/data_enem0.csv")



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916 entries, 0 to 915
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   enem_version  916 non-null    object
 1   header        916 non-null    object
 2   statement     916 non-null    object
 3   CE            916 non-null    object
 4   DS            916 non-null    object
 5   EK            916 non-null    object
 6   IC            916 non-null    object
 7   MR            916 non-null    object
 8   TC            916 non-null    object
 9   id            916 non-null    object
 10  image         916 non-null    object
 11  gab           916 non-null    object
 12  A             916 non-null    object
 13  B             916 non-null    object
 14  C             916 non-null    object
 15  D             916 non-null    object
 16  E             916 non-null    object
dtypes: object(17)
memory usage: 121.8+ KB

#############
# Questão 0 #
#############

Ano/Versão

In [ ]:
bd_enem_long = pd.read_csv("/content/drive/MyDrive/unicamp - IA024 /projeto_enem/data_enem_long.csv") 
bd_enem_long['text'][0]

bd_enem_long = bd_enem_long.sample(frac=1).reset_index(drop=True)

bd_enem_long

,...1,enem_version,header,statement,CE,DS,EK,IC,MR,TC,id,image,gab,question_id,opcao,opcao_text,text,label,train_test
0,972,2014-2,A última edição deste periódico apresenta mais...,Esse editorial faz uma leitura diferenciada de...,No,No,No,No,No,Yes,99,No,D,2014-2_99,D,interpretar criticamente fatos noticiados e co...,A última edição deste periódico apresenta mais...,1,1
1,925,2014-1,"Na década de 1940, na Região Centro-Oest...",Para suprir o déficit nutricional a que os pro...,No,Yes,Yes,No,No,Yes,47,No,D,2014-1_47,D,aminoácidos essenciais.,"Na década de 1940, na Região Centro-Oest...",1,1
2,483,2011-2,O que é possível dizer em 140 caracteres?Suces...,"O Twitter se presta a diversas finalidades, en...",No,No,Yes,No,No,Yes,135,No,E,2011-2_135,B,constitui recurso próprio para a aquisição da ...,O que é possível dizer em 140 caracteres?Suces...,0,1
3,1464,2016_2_-1,Até 1824 acreditava-se que as máquinas térmica...,Tal limitação ocorre porque essas máquinas,No,No,Yes,No,No,Yes,61,No,B,2016_2_-1_61,C,utilizam transformações adiabáticas.,Até 1824 acreditava-se que as máquinas térmica...,0,0
4,1452,2016_2_-1,Um jovem suspeita que não é filho bioló...,A condição genotípica que possibilita que...,No,Yes,No,No,No,Yes,47,No,A,2016_2_-1_47,A,o pai e a mãe sejam heterozigotos para ...,Um jovem suspeita que não é filho bioló...,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7323,1590,2017-1,Quer suporte paraencontrar pessoasque comparti...,"O consumidor do século XXI, chamado de ...",No,No,No,No,No,Yes,17,Yes,A,2017-1_17,B,busca constante pelo menor preço.,Quer suporte paraencontrar pessoasque comparti...,0,0
7324,783,2013-1,Estudos de fluxo de energia em ecossiste...,"Dos grupos de seres vivos citados, os que cont...",No,Yes,No,No,No,No,80,No,B,2013-1_80,D,insetos.,Estudos de fluxo de energia em ecossiste...,0,1
7325,899,2014-1,Estatuto da Frente Negra Brasileira (FNB)Art. ...,Quando foi fechada pela ditadura do Esta...,No,No,Yes,No,No,Yes,21,No,A,2014-1_21,A,"política, engajada na luta por direitos sociai...",Estatuto da Frente Negra Brasileira (FNB)Art. ...,1,1
7326,448,2011-2,TEXTO IVocê tem palacete reluzenteTem joias e ...,Um texto pertencente ao patrimônio literá...,No,No,No,No,No,Yes,100,No,A,2011-2_100,C,da maldade do povo a perguntar sobre a honesti...,TEXTO IVocê tem palacete reluzenteTem joias e ...,0,1


In [ ]:
df_train = bd_enem_long.query('train_test ==1')[['text', 'label']]
df_train.columns = ['sentence', 'label'] 

df_test = bd_enem_long.query('train_test ==0')[['text', 'label']]
df_test.columns = ['sentence', 'label'] 

df_train["context_size"] = df_train["sentence"].apply(lambda x: len(x.split(" ")))
df_test["context_size"] = df_test["sentence"].apply(lambda x: len(x.split(" ")))

df_train = df_train.query('context_size <=510')[['sentence', 'label']] 
df_test = df_test.query('context_size <=510')[['sentence', 'label']] 

df_train.to_csv('df_train.csv', index=None)
df_test.to_csv('df_test.csv', index=None)

In [ ]:
!head df_train.csv

sentence,label
"A última edição deste periódico apresenta mais uma vez  tema relacionado ao tratamento dado ao lixo caseiro, aquele que produzimos no dia a dia. A informação agora passa pelo problema do material jogado na estrada vicinal que  liga  o  município  de  Rio  Claro  ao  distrito  de  Ajapi. Infelizmente, no local em questão, a reportagem encontrou mais  uma  forma  errada  de  destinação  do  lixo:  material atirado ao lado da pista como se  isso fosse o ideal. Muitos moradores, por exemplo, retiram o lixo de suas residências e,  em  vez  de  um  destino  correto,  procuram  dispensá-lo em outras regiões. Uma situação no mínimo incômoda. Se você sai de casa para jogar o lixo em outra localidade, por que não o fazer no local ideal? É muita falta de educação achar que aquilo que não é correto para sua região possa ser para outra. A reciclagem do lixo doméstico é um passo inteligente e de consciência. Olha o exemplo que passamos aos mais jovens! Quem aprende errado coloca em p

In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset(
    'csv',
     data_files={'train': 'df_train.csv',
                 'test': 'df_test.csv'
     }
 )
raw_dataset

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-1e157addff2c278c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 5850
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 1456
    })
})

In [ ]:
checkpoint = 'neuralmind/bert-base-portuguese-cased'

tokenizer = AutoTokenizer.from_pretrained(checkpoint,do_lower_case=False)

context_length =  510 # 512 - [CLS] [SEP]


def tokenize_fn(batch):
  return tokenizer(batch['sentence'], truncation=True, max_length=context_length)
  
tokenized_datasets = raw_dataset.map(tokenize_fn, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2)


#tokenized_datasets
#df = pd.DataFrame(tokenized_datasets['test'])
#df['context_size'] = df['input_ids'].apply(lambda x: len(x))

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

In [ ]:
summary(model)

Layer (type:depth-idx)                                  Param #
BertForSequenceClassification                           --
├─BertModel: 1-1                                        --
│    └─BertEmbeddings: 2-1                              --
│    │    └─Embedding: 3-1                              22,881,792
│    │    └─Embedding: 3-2                              393,216
│    │    └─Embedding: 3-3                              1,536
│    │    └─LayerNorm: 3-4                              1,536
│    │    └─Dropout: 3-5                                --
│    └─BertEncoder: 2-2                                 --
│    │    └─ModuleList: 3-6                             85,054,464
│    └─BertPooler: 2-3                                  --
│    │    └─Linear: 3-7                                 590,592
│    │    └─Tanh: 3-8                                   --
├─Dropout: 1-2                                          --
├─Linear: 1-3                                           1,538
Total params: 10

In [ ]:
training_args = TrainingArguments(
  output_dir='training_dir',
  evaluation_strategy='epoch',
  save_strategy='epoch',
  num_train_epochs=3,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=64,
)

In [ ]:
def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  acc = np.mean(predictions == labels)
  f1 = f1_score(labels, predictions, average='micro')
  return {'accuracy': acc, 'f1': f1}

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
!ls training_dir

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint,do_lower_case=False)

context_length =  510 # 512 - [CLS] [SEP]

def tokenize_fn(batch):
  return tokenizer(batch['sentence'], truncation=True, max_length=context_length)
  
tokenized_datasets = raw_dataset.map(tokenize_fn, batched=True)

savedmodel  = pipeline(
    task='text-classification',
    model='training_dir/checkpoint-1098', 
    tokenizer=tokenizer
    )

In [ ]:
test_pred = savedmodel(tokenized_datasets['test']['sentence'], truncation=True, max_length=context_length)

#savedmodel()

In [ ]:
def get_label(d):
  return int(d['label'].split('_')[1])

test_pred2 = [get_label(d) for d in test_pred]

print("acc:", accuracy_score(raw_dataset['test']['label'], test_pred2))
print("f1:", f1_score(raw_dataset['test']['label'], test_pred2, average='micro'))

In [ ]:
# Scikit-Learn is transitioning to V1 but it's not available on Colab
# The changes modify how confusion matrices are plotted
def plot_cm(cm):
  classes = ['0', '1']
  df_cm = pd.DataFrame(cm, index=classes, columns=classes)
  ax = sn.heatmap(df_cm, annot=True, fmt='g')
  ax.set_xlabel("Predicted")
  ax.set_ylabel("Target")

cm = confusion_matrix(tokenized_datasets['test']['label'], test_pred2, normalize='true')
plot_cm(cm)



In [ ]:
type(test_pred)

test_results_bertimbau_base =  pd.DataFrame( test_pred)
test_results_bertimbau_base["true_label"] = tokenized_datasets['test']['label']

test_results_bertimbau_base.head()

test_results_bertimbau_base.to_csv("/content/drive/MyDrive/unicamp - IA024 /projeto_enem/test_results_bertimbau_base.csv")
